In [1]:
import pandas as pd
pd.set_option("display.max_columns", None)

import numpy as np
from pathlib import Path
import re

from common import *

<h1><span style="color:red">
    WARNING:</span> This file is meant to be executed top to bottom. 
</h1><h2>There are quite a few parts copy & pasted. Hence, MANY variables are shadowed.</h2>

# Step 0. Data Production: Each of these cells takes 14 minutes to execute on Stefan's machine

In [2]:
DATA_PREFIX = "last"
THRESHOLD = 0.000001  # 1.0 and 0.9999999... are the same, right?

raw_results_path = Path("/Volumes/Transcend/data/QRefactoring-results-final/QRefactoring-results-final")

OL_file_path = Path(f"{DATA_PREFIX.replace('%', '')}_vals_OL.pkl")
OED_file_path = Path(f"{DATA_PREFIX.replace('%', '')}_vals_OED.pkl")

paper_tables_dir = Path("/Users/stefan/Library/CloudStorage/Dropbox/Apps/Overleaf/ASE2023 - QRepair/generated")

In [3]:
# This is a helper!
def extract_best_solutions(files, earliest_finish=None):
    if earliest_finish == "last":  # override last
        earliest_finish = None
    
    earliest_finish_times = {}
    if earliest_finish:
        for earliest_finish_file in list(raw_results_path.glob("*earliest_finish.csv")):
            problem = earliest_finish_file.stem.replace("_earliest_finish", "")

            earliest_finish_df = pd.read_csv(earliest_finish_file)
            earliest_finish_time = earliest_finish_df.iloc[0][earliest_finish]
            earliest_finish_times[problem] = earliest_finish_time
    # print("Earliest finish times:")
    # pprint(earliest_finish_times)
    best_OL_rows = []
    best_OED_rows = []
    # extract last gen value
    for pi_file in sorted(files):
        problem, seed, option = extract_info_from_file(pi_file)
        qubits, arbitrary = QUBITS_and_ARBITRARY[problem]
        
        last_row = dict(problem=problem, option=option, seed=seed, qubits=qubits, arbitrary=arbitrary)
        results_file_df = pd.read_csv(pi_file)
        if earliest_finish:
            if problem not in earliest_finish_times:
                print("WARNING!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
                print("No earliest finish time for problem", problem)
            else:
                results_file_df = results_file_df[results_file_df.timestamp <= earliest_finish_df[earliest_finish].iloc[0]]
        
        last_gen_df = results_file_df[results_file_df.ngen == results_file_df.ngen.max()].reset_index()
        last_gen_df["no_error_prob_actual"] = 1 - last_gen_df.apply(lambda row: get_actual_error_rate(row.num_gates, row.num_nonloc_gates), axis=1)
        last_gen_df["OED"] = last_gen_df.overlap * last_gen_df.no_error_prob_actual
        
        best_OL_row = dict(last_row)
        best_OL_row.update(last_gen_df.sort_values("overlap", ascending=False).iloc[0].to_dict())
        best_OL_rows.append(best_OL_row)
        
        best_OED_row = dict(last_row)
        best_OED_row.update(last_gen_df.sort_values("OED", ascending=False).iloc[0].to_dict())
        best_OED_rows.append(best_OED_row)

    OL_df = pd.DataFrame(best_OL_rows).sort_values(by=["problem", "option", "seed"])
    OED_df = pd.DataFrame(best_OED_rows).sort_values(by=["problem", "option", "seed"])
    
    return OL_df, OED_df

In [4]:
%%time
if OL_file_path.exists() and OED_file_path.exists():
    print("Found data files. No need to extract.")
else:
    print("Data files not found. Starting the data extraction.")
    
    output_files = list(raw_results_path.glob("*.csv"))
    output_files = [f for f in output_files if "logbook" not in str(f)]
    output_files = [f for f in output_files if "seed" in str(f)]
    output_files = [f for f in output_files if "PI.csv" not in str(f)]
    output_files = [f for f in output_files if "DCI.csv" not in str(f)]
    output_files = [f for f in output_files if "HVrefpoint.csv" not in str(f)]
    output_files = [f for f in output_files if "globalPareto.csv" not in str(f)]
    output_files = [f for f in output_files if "earliest_finish.csv" not in str(f)]
          
          
    OL_df, OED_df = extract_best_solutions(output_files, earliest_finish=DATA_PREFIX)
    OL_df.to_pickle(OL_file_path)
    OED_df.to_pickle(OED_file_path)

Found data files. No need to extract.
CPU times: user 2.47 ms, sys: 800 µs, total: 3.27 ms
Wall time: 1.48 ms


# RQ 1.1 & 2.1 (Theoretical gains)

In [5]:
all_vals_df = pd.read_pickle(OL_file_path)
all_vals_df["repair"] = all_vals_df.problem.apply(lambda p: p in repair_circuits)

In [6]:
def _check_improvement(row): 
    ref_num_gates, ref_depth, ref_non_local = reference_fitness_values[row.problem]
    # either all equal, or some better, some worse...
    if (row.num_gates == ref_num_gates and row.depth == ref_depth and row.num_nonloc_gates == ref_non_local) or \
       ((row.num_gates > ref_num_gates or row.depth > ref_depth or row.num_nonloc_gates > ref_non_local) and \
        (row.num_gates < ref_num_gates or row.depth < ref_depth or row.num_nonloc_gates < ref_non_local)):
        return "Pareto Equal"
    
    # not all equal, not pareto_equal, so it's either better or worse
    if row.num_gates >= ref_num_gates and row.depth >= ref_depth and row.num_nonloc_gates >= ref_non_local:
        return "Worse"
    
    # one of them is better
    if row.num_gates <= ref_num_gates and row.depth <= ref_depth and row.num_nonloc_gates <= ref_non_local:
        return "Optimized"

def get_operator_categorisation_OL(row):   
    if (1.0 - row.overlap) > THRESHOLD:  # TODO: Check if this is the overlap? ...
        return "Faulty"
    
    # it's not buggy, so check if we are better or not...
    return _check_improvement(row) 
    
def get_operator_categorisation_OED(row):
    ref_num_gates, ref_depth, ref_non_local = reference_fitness_values[row.problem]
    ref_OED_actual = 1 - get_actual_error_rate(ref_num_gates, ref_non_local)
    
    if (ref_OED_actual - row.OED) > THRESHOLD:
        return "Faulty"
    
    # it's not buggy, so check if we are better or not...
    return _check_improvement(row)

## RQ 1.1 - Repair (theoretical)? 

In [7]:
repair_df = all_vals_df[all_vals_df.repair].reset_index()
repair_df["Categorisation_OL"] = repair_df.apply(get_operator_categorisation_OL, axis=1)
# repair_df["Categorisation_OED"] = repair_df.apply(get_operator_categorisation_OED, axis=1)

In [8]:
display(repair_df.groupby("Categorisation_OL").count())
# display(repair_df.groupby("Categorisation_OED").count())

,level_0,problem,option,seed,qubits,arbitrary,index,overlap,num_gates,depth,num_nonloc_gates,num_parameters,ngen,neval,timestamp,HV,no_error_prob_actual,OED,repair
Categorisation_OL,,,,,,,,,,,,,,,,,,,
Faulty,337,337,337,337,337,337,337,337,337,337,337,337,337,337,337,0,337,337,337
Optimized,954,954,954,954,954,954,954,954,954,954,954,954,954,954,954,0,954,954,954
Pareto Equal,204,204,204,204,204,204,204,204,204,204,204,204,204,204,204,0,204,204,204
Worse,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,0,125,125,125


In [9]:
tables = {}
for opt in [HYBRID, NONHYBRID, FIXED]:
    print(better_options[opt])
    rows = []
    for problem in repair_df.problem.unique():
        qubits=repair_df[repair_df.problem == problem].qubits.iloc[0]
        arbitrary = repair_df[repair_df.problem == problem].arbitrary.iloc[0]
        row = dict(problem = problem, qubits=qubits, Arbitrary="Arbitrary" if arbitrary else "Specific" )
        
        for which in ["Optimized", "Pareto Equal", "Worse", "Faulty"]:
            row[which] = len(repair_df[
                (repair_df.option == opt) & 
                (repair_df.problem == problem) & 
                (repair_df.Categorisation_OL == which)                
            ])
        rows.append(row)
    tab = pd.DataFrame(rows)
    # tab = tab.reindex(tab.problem)
    tables[opt] = tab
    # display(tab)

# sum_table = tables[HYBRID].applymap(lambda v: f"{v}").add(tables[NONHYBRID].applymap(lambda v: f" / {v}")).add(tables[FIXED].applymap(lambda v: f" / {v}"))
# sum_table["problem"] = sum_table.problem.apply(lambda v: v.split("/")[0].strip())
# sum_table["Qubits"] = sum_table.qubits.apply(lambda v: v.split("/")[0].strip() + " qubits")
# sum_table["Arbitrary"] = sum_table.Arbitrary.apply(lambda v: v.split("/")[0].strip())

# sum_table["Problem"] = "[" + sum_table["Arbitrary"] + "]" + sum_table["problem"]
# sum_table = sum_table.sort_values(by=["Qubits", "problem"], key=lambda col: col.str.lower())
# sum_table.index = pd.MultiIndex.from_frame(sum_table[["Qubits", "Problem"]])
# sum_table = sum_table.drop(columns=["problem", "qubits", "Arbitrary", "Problem", "Qubits"])


sum_table = tables[HYBRID].applymap(lambda v: f"{v}").add(tables[NONHYBRID].applymap(lambda v: f" / {v}")).add(tables[FIXED].applymap(lambda v: f" / {v}"))
sum_table["problem"] = sum_table.problem.apply(lambda v: v.split("/")[0].strip())
sum_table["Qubits"] = sum_table.qubits.apply(lambda v: v.split("/")[0].strip() + " qubits")
sum_table["Input State"] = sum_table.Arbitrary.apply(lambda v: v.split("/")[0].strip())

sum_table["Problem"] =  sum_table["problem"] + " (" + sum_table["Qubits"]+")"
sum_table = sum_table.sort_values(by=["Arbitrary", "qubits"], ascending=[False,True], key=lambda col: col.str.lower())
sum_table.index = pd.MultiIndex.from_frame(sum_table[["Input State", "Problem"]])
sum_table = sum_table.drop(columns=["problem", "qubits", "Arbitrary", "Problem", "Qubits", "Input State"])

# RQ11_table = sum_table.set_index("Problem").sort_values(by=["qubits", "problem"], key=lambda col: col.str.lower()).drop(columns=["problem", "Qubits", "Arbitrary"])
RQ11_table = sum_table
RQ11_table

Hybrid
NonHybrid
Fixed


Optimized Pareto Equal       Worse   
Input State Problem                                                    
Specific    QG_8 (2 qubits)    30 / 30 / 30    0 / 0 / 0   0 / 0 / 0  \
            QSO_6 (2 qubits)     26 / 0 / 0  4 / 30 / 30   0 / 0 / 0   
            QSO_5 (3 qubits)   30 / 30 / 30    0 / 0 / 0   0 / 0 / 0   
            QSE_15 (4 qubits)  30 / 30 / 30    0 / 0 / 0   0 / 0 / 0   
            QSE_3 (5 qubits)   30 / 30 / 30    0 / 0 / 0   0 / 0 / 0   
Arbitrary   QSE2_2 (2 qubits)    30 / 0 / 0   0 / 0 / 30  0 / 16 / 0   
            QSE2_3 (3 qubits)     0 / 0 / 0   17 / 0 / 0   8 / 0 / 8   
            QSE2_4 (4 qubits)     0 / 0 / 0    0 / 0 / 0   7 / 0 / 0   
            QSE2_5 (5 qubits)     0 / 0 / 0    0 / 0 / 0   6 / 0 / 0   

                                     Faulty  
Input State Problem                          
Specific    QG_8 (2 qubits)       0 / 0 / 0  
            QSO_6 (2 qubits)      0 / 0 / 0  
            QSO_5 (3 qubits)      0 / 0 / 0  
            QSE_15 (4 qubits)     0 / 0 / 0  
            QSE_3 (5 qubits)      0 / 0 / 0  
Arbitrary   QSE2_2 (2 qubits)    0 / 14 / 0  
            QSE2_3 (3 qubits)   5 / 30 / 22  
            QSE2_4 (4 qubits)  23 / 30 / 30  
            QSE2_5 (5 qubits)  24 / 30 / 30

## Calculate Relative Improvement

In [10]:
interesting_cols = ["num_gates", "depth", "num_nonloc_gates", "num_parameters"]
repair_df[repair_df.Categorisation_OL == "Optimized"][interesting_cols]

,num_gates,depth,num_nonloc_gates,num_parameters
0,2.0,1.0,0.0,0.0
1,2.0,1.0,0.0,0.0
2,2.0,1.0,0.0,0.0
3,2.0,1.0,0.0,0.0
4,2.0,1.0,0.0,0.0
...,...,...,...,...
1585,2.0,2.0,1.0,1.0
1586,2.0,2.0,1.0,1.0
1587,2.0,2.0,1.0,1.0
1588,2.0,2.0,1.0,2.0


## RQ2.1 - Optimization (theoretical)

In [11]:
optimization_df = all_vals_df[all_vals_df.repair == False].reset_index()
optimization_df["Categorisation_OL"] = optimization_df.apply(get_operator_categorisation_OL, axis=1)

In [12]:
tables = {}
for option in [HYBRID, NONHYBRID, FIXED]:
    print(better_options[option])
    rows = []
    for problem in optimization_df.problem.unique():
        # print(problem)
        qubits=optimization_df[optimization_df.problem == problem].qubits.iloc[0]
        arbitrary = optimization_df[optimization_df.problem == problem].arbitrary.iloc[0]
        row = dict(problem = problem, qubits=qubits, Arbitrary=arbitrary)

        for which in ["Optimized", "Pareto Equal", "Worse", "Faulty"]:
            row[which] = len(optimization_df[
                (optimization_df.option == option) & 
                (optimization_df.problem == problem) & 
                (optimization_df.Categorisation_OL == which)                
            ])
        rows.append(row)
    tab = pd.DataFrame(rows)
    tables[option] = tab
    # display(tab)

sum_table = tables[HYBRID].applymap(lambda v: f"{v}").add(tables[NONHYBRID].applymap(lambda v: f" / {v}")).add(tables[FIXED].applymap(lambda v: f" / {v}"))
sum_table["problem"] = sum_table.problem.apply(lambda v: v.split("/")[0].strip())
sum_table["Qubits"] = sum_table.qubits.apply(lambda v: v.split("/")[0].strip() + " qubits")
sum_table["Arbitrary"] = sum_table.Arbitrary.apply(lambda v: v.split("/")[0].strip())

sum_table["Problem"] = "[" + sum_table["Arbitrary"] + "]" + sum_table["problem"]
sum_table = sum_table.sort_values(by=["Qubits", "problem"], key=lambda col: col.str.lower())
sum_table.index = pd.MultiIndex.from_frame(sum_table[["Qubits", "Problem"]])
sum_table = sum_table.drop(columns=["problem", "qubits", "Arbitrary", "Problem", "Qubits"])

# RQ11_table = sum_table.set_index("Problem").sort_values(by=["qubits", "problem"], key=lambda col: col.str.lower()).drop(columns=["problem", "Qubits", "Arbitrary"])
RQ21_table = sum_table
RQ21_table

Hybrid
NonHybrid
Fixed


Optimized  Pareto Equal   
Qubits   Problem                                                      
2 qubits [True]AA2                        30 / 6 / 30     0 / 0 / 0  \
         [False]GHZ2                        0 / 0 / 0  30 / 30 / 30   
         [True]hamiltonian_simulation_2    30 / 7 / 0     0 / 0 / 0   
         [True]iswap_n2                  30 / 30 / 30     0 / 0 / 0   
         [True]QFT2                       12 / 28 / 0     7 / 0 / 0   
         [True]quantum_walk                 3 / 0 / 0     3 / 0 / 0   
         [False]wstate2                  30 / 30 / 30     0 / 0 / 0   
3 qubits [True]AA3                         15 / 0 / 0    11 / 0 / 0   
         [True]fredkin_n3                   6 / 0 / 0     7 / 0 / 0   
         [False]GHZ3                      30 / 6 / 11    0 / 13 / 1   
         [False]GS3                        28 / 5 / 0     1 / 7 / 1   
         [True]hamiltonian_simulation_3    30 / 0 / 0     0 / 0 / 0   
         [False]linearsolver_n3            30 / 6 / 0     0 / 0 / 0   
         [True]QFT3                         0 / 0 / 0     0 / 0 / 0   
         [True]quantum_mc_F                13 / 0 / 0     2 / 0 / 0   
         [True]teleportation_n3            18 / 0 / 0     5 / 0 / 0   
         [True]tofolli_n3                  18 / 0 / 0    11 / 0 / 0   
         [False]wstate3                    30 / 0 / 0     0 / 0 / 0   
         [False]wstate_n3                  30 / 0 / 0     0 / 0 / 0   
4 qubits [True]AA4                          0 / 0 / 0     0 / 0 / 0   
         [True]adder_n4                     3 / 0 / 0     0 / 0 / 0   
         [False]bell_n4                    30 / 0 / 0     0 / 0 / 0   
         [False]cat_state_n4                5 / 2 / 1     3 / 0 / 0   
         [False]GHZ4                        5 / 2 / 1     3 / 0 / 0   
         [False]GS4                        13 / 1 / 0     6 / 2 / 0   
         [True]hamiltonian_simulation_4    23 / 0 / 0     1 / 0 / 0   
         [True]hs4_n4                      24 / 1 / 0     6 / 0 / 0   
         [True]QFT4                         0 / 0 / 0     0 / 0 / 0   
         [False]qrng_n4                     0 / 0 / 0  27 / 30 / 30   
         [False]wstate4                    17 / 0 / 0     9 / 0 / 0   
5 qubits [True]AA5                          0 / 0 / 0     0 / 0 / 0   
         [False]GHZ5                        0 / 0 / 0     0 / 0 / 0   
         [False]GS5                         2 / 0 / 0     0 / 0 / 0   
         [True]hamiltonian_simulation_5    12 / 0 / 0     1 / 0 / 0   
         [False]lpn_n5                    23 / 11 / 2     3 / 2 / 2   
         [True]qec_en_n5                    0 / 0 / 0     0 / 0 / 0   
         [True]QFT5                         0 / 0 / 0     0 / 0 / 0   
         [False]wstate5                     1 / 0 / 0     7 / 0 / 0   

                                              Worse        Faulty  
Qubits   Problem                                                   
2 qubits [True]AA2                        0 / 0 / 0    0 / 24 / 0  
         [False]GHZ2                      0 / 0 / 0     0 / 0 / 0  
         [True]hamiltonian_simulation_2   0 / 0 / 0   0 / 23 / 30  
         [True]iswap_n2                   0 / 0 / 0     0 / 0 / 0  
         [True]QFT2                      10 / 0 / 0    1 / 2 / 30  
         [True]quantum_walk               9 / 0 / 0  15 / 30 / 30  
         [False]wstate2                   0 / 0 / 0     0 / 0 / 0  
3 qubits [True]AA3                        0 / 0 / 0   4 / 30 / 30  
         [True]fredkin_n3                 5 / 0 / 0  12 / 30 / 30  
         [False]GHZ3                     0 / 6 / 13     0 / 5 / 5  
         [False]GS3                       1 / 1 / 1   0 / 17 / 28  
         [True]hamiltonian_simulation_3   0 / 0 / 0   0 / 30 / 30  
         [False]linearsolver_n3           0 / 0 / 0   0 / 24 / 30  
         [True]QFT3                       2 / 0 / 0  28 / 30 / 30  
         [True]quantum_mc_F               1 / 0 / 0  14 / 30 / 30  
         [True]teleportation_n3           0 / 0

## Create Summary Tables (because the full table is too long!)

In [13]:
tables = {}
for option in [HYBRID, NONHYBRID, FIXED]:
    print(better_options[option])
    rows = []
    for problem in optimization_df.problem.unique():
        # print(problem)
        qubits=optimization_df[optimization_df.problem == problem].qubits.iloc[0]
        arbitrary = optimization_df[optimization_df.problem == problem].arbitrary.iloc[0]
        if arbitrary:
            arb = "Arbitrary"
        else:
            arb = "Specific"
        row = dict(problem = problem, qubits=qubits, Arbitrary=arb)

        for which in ["Optimized", "Pareto Equal", "Worse", "Faulty"]:
            row[which] = len(optimization_df[
                (optimization_df.option == option) & 
                (optimization_df.problem == problem) & 
                (optimization_df.Categorisation_OL == which)                
            ])
        rows.append(row)
    tab = pd.DataFrame(rows)
    tables[option] = tab
    
    
    tab = tab.sum().drop(columns=["problem", "qubits"])
    tables[option] = pd.DataFrame(tab)
        
sum_table = tables[HYBRID].applymap(lambda v: f"{v}").add(tables[NONHYBRID].applymap(lambda v: f" / {v}")).add(tables[FIXED].applymap(lambda v: f" / {v}"))

RQ21_summary_table_sum = sum_table.T.drop(columns=["qubits", "problem", "Arbitrary"])
RQ21_summary_table_sum

Hybrid
NonHybrid
Fixed


,Optimized,Pareto Equal,Worse,Faulty
0,541 / 135 / 105,143 / 84 / 64,50 / 10 / 14,406 / 911 / 957


In [14]:
tables = {}
for option in [HYBRID, NONHYBRID, FIXED]:
    print(better_options[option])
    rows = []
    for problem in optimization_df.problem.unique():
        # print(problem)
        qubits=optimization_df[optimization_df.problem == problem].qubits.iloc[0]
        arbitrary = optimization_df[optimization_df.problem == problem].arbitrary.iloc[0]
        if arbitrary:
            arb = "Arbitrary"
        else:
            arb = "Specific"
        row = dict(problem = problem, qubits=qubits, Arbitrary=arb)

        for which in ["Optimized", "Pareto Equal", "Worse", "Faulty"]:
            row[which] = len(optimization_df[
                (optimization_df.option == option) & 
                (optimization_df.problem == problem) & 
                (optimization_df.Categorisation_OL == which)                
            ])
        rows.append(row)
    tab = pd.DataFrame(rows)
    tables[option] = tab
    tab = tab.groupby("Arbitrary").sum().drop(columns=["problem", "qubits"])
    tables[option] = tab
    
sum_table = tables[HYBRID].applymap(lambda v: f"{v}").add(tables[NONHYBRID].applymap(lambda v: f" / {v}")).add(tables[FIXED].applymap(lambda v: f" / {v}"))

RQ21_summary_table_input_state = sum_table
RQ21_summary_table_input_state

Hybrid
NonHybrid
Fixed


,Optimized,Pareto Equal,Worse,Faulty
Arbitrary,,,,
Arbitrary,267 / 72 / 60,54 / 0 / 0,27 / 0 / 0,282 / 558 / 570
Specific,274 / 63 / 45,89 / 84 / 64,23 / 10 / 14,124 / 353 / 387


In [15]:
tables = {}
for option in [HYBRID, NONHYBRID, FIXED]:
    print(better_options[option])
    rows = []
    for problem in optimization_df.problem.unique():
        # print(problem)
        qubits=optimization_df[optimization_df.problem == problem].qubits.iloc[0]
        arbitrary = optimization_df[optimization_df.problem == problem].arbitrary.iloc[0]
        if arbitrary:
            arb = "Arbitrary"
        else:
            arb = "Specific"
        row = dict(problem = problem, qubits=qubits, Arbitrary=arb)

        for which in ["Optimized", "Pareto Equal", "Worse", "Faulty"]:
            row[which] = len(optimization_df[
                (optimization_df.option == option) & 
                (optimization_df.problem == problem) & 
                (optimization_df.Categorisation_OL == which)                
            ])
        rows.append(row)
    tab = pd.DataFrame(rows)
    tables[option] = tab
    
    tab = tab.groupby("qubits").sum().drop(columns=["problem", "Arbitrary"])
    tables[option] = tab
        
sum_table = tables[HYBRID].applymap(lambda v: f"{v}").add(tables[NONHYBRID].applymap(lambda v: f" / {v}")).add(tables[FIXED].applymap(lambda v: f" / {v}"))

RQ21_summary_table_qubits = sum_table
RQ21_summary_table_qubits

Hybrid
NonHybrid
Fixed


,Optimized,Pareto Equal,Worse,Faulty
qubits,,,,
2,135 / 101 / 90,40 / 30 / 30,19 / 0 / 0,16 / 79 / 90
3,248 / 17 / 11,37 / 20 / 2,9 / 7 / 14,66 / 316 / 333
4,120 / 6 / 2,55 / 32 / 30,14 / 3 / 0,141 / 289 / 298
5,38 / 11 / 2,11 / 2 / 2,8 / 0 / 0,183 / 227 / 236


# RQ 1.2 & 2.2 (Practical gains)

In [16]:
all_vals_df = pd.read_pickle(OED_file_path)
all_vals_df["repair"] = all_vals_df.problem.apply(lambda p: p in repair_circuits)

## RQ 1.1 - Repair (practical)? 

In [17]:
repair_df = all_vals_df[all_vals_df.repair].reset_index()      
repair_df["Categorisation_OED"] = repair_df.apply(get_operator_categorisation_OED, axis=1)

In [18]:
display(repair_df.groupby("Categorisation_OED").count())

,level_0,problem,option,seed,qubits,arbitrary,index,overlap,num_gates,depth,num_nonloc_gates,num_parameters,ngen,neval,timestamp,HV,no_error_prob_actual,OED,repair
Categorisation_OED,,,,,,,,,,,,,,,,,,,
Faulty,324,324,324,324,324,324,324,324,324,324,324,324,324,324,324,0,324,324,324
Optimized,1074,1074,1074,1074,1074,1074,1074,1074,1074,1074,1074,1074,1074,1074,1074,0,1074,1074,1074
Pareto Equal,219,219,219,219,219,219,219,219,219,219,219,219,219,219,219,0,219,219,219
Worse,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,0,3,3,3


In [19]:
tables = {}
for opt in [HYBRID, NONHYBRID, FIXED]:
    print(better_options[opt])
    rows = []
    for problem in repair_df.problem.unique():
        qubits=repair_df[repair_df.problem == problem].qubits.iloc[0]
        arbitrary = repair_df[repair_df.problem == problem].arbitrary.iloc[0]
        row = dict(problem = problem, qubits=qubits, Arbitrary="Arbitrary" if arbitrary else "Specific")
        
        for which in ["Optimized", "Pareto Equal", "Worse", "Faulty"]:
            row[which] = len(repair_df[
                (repair_df.option == opt) & 
                (repair_df.problem == problem) & 
                (repair_df.Categorisation_OED == which)                
            ])
        rows.append(row)
    tab = pd.DataFrame(rows)
    tables[opt] = tab

sum_table = tables[HYBRID].applymap(lambda v: f"{v}").add(tables[NONHYBRID].applymap(lambda v: f" / {v}")).add(tables[FIXED].applymap(lambda v: f" / {v}"))
sum_table["problem"] = sum_table.problem.apply(lambda v: v.split("/")[0].strip())
sum_table["Qubits"] = sum_table.qubits.apply(lambda v: v.split("/")[0].strip() + " qubits")
sum_table["Input State"] = sum_table.Arbitrary.apply(lambda v: v.split("/")[0].strip())

sum_table["Problem"] =  sum_table["problem"] + " (" + sum_table["Qubits"]+")"
sum_table = sum_table.sort_values(by=["Arbitrary", "qubits"], ascending=[False,True], key=lambda col: col.str.lower())
sum_table.index = pd.MultiIndex.from_frame(sum_table[["Input State", "Problem"]])
sum_table = sum_table.drop(columns=["problem", "qubits", "Arbitrary", "Problem", "Qubits", "Input State"])

RQ12_table = sum_table
RQ12_table

Hybrid
NonHybrid
Fixed


Optimized Pareto Equal      Worse   
Input State Problem                                                   
Specific    QG_8 (2 qubits)    30 / 30 / 30    0 / 0 / 0  0 / 0 / 0  \
            QSO_6 (2 qubits)     26 / 0 / 0  4 / 30 / 30  0 / 0 / 0   
            QSO_5 (3 qubits)   30 / 30 / 30    0 / 0 / 0  0 / 0 / 0   
            QSE_15 (4 qubits)  30 / 30 / 30    0 / 0 / 0  0 / 0 / 0   
            QSE_3 (5 qubits)   30 / 30 / 30    0 / 0 / 0  0 / 0 / 0   
Arbitrary   QSE2_2 (2 qubits)    30 / 0 / 0   0 / 0 / 30  0 / 0 / 0   
            QSE2_3 (3 qubits)    14 / 0 / 0   14 / 0 / 0  0 / 0 / 0   
            QSE2_4 (4 qubits)     1 / 0 / 0    3 / 0 / 0  1 / 0 / 0   
            QSE2_5 (5 qubits)     0 / 0 / 0    6 / 0 / 0  0 / 0 / 0   

                                     Faulty  
Input State Problem                          
Specific    QG_8 (2 qubits)       0 / 0 / 0  
            QSO_6 (2 qubits)      0 / 0 / 0  
            QSO_5 (3 qubits)      0 / 0 / 0  
            QSE_15 (4 qubits)     0 / 0 / 0  
            QSE_3 (5 qubits)      0 / 0 / 0  
Arbitrary   QSE2_2 (2 qubits)    0 / 30 / 0  
            QSE2_3 (3 qubits)   2 / 30 / 30  
            QSE2_4 (4 qubits)  25 / 30 / 30  
            QSE2_5 (5 qubits)  24 / 30 / 30

## RQ2.1 - Optimization (practical)

In [20]:
optimization_df = all_vals_df[all_vals_df.repair == False].reset_index()
optimization_df["Categorisation_OED"] = optimization_df.apply(get_operator_categorisation_OED, axis=1)

In [21]:
tables = {}
for option in [HYBRID, NONHYBRID, FIXED]:
    print(better_options[option])
    rows = []
    for problem in optimization_df.problem.unique():
        # print(problem)
        qubits=optimization_df[optimization_df.problem == problem].qubits.iloc[0]
        arbitrary = optimization_df[optimization_df.problem == problem].arbitrary.iloc[0]
        row = dict(problem = problem, qubits=qubits, Arbitrary=arbitrary)

        for which in ["Optimized", "Pareto Equal", "Worse", "Faulty"]:
            row[which] = len(optimization_df[
                (optimization_df.option == option) & 
                (optimization_df.problem == problem) & 
                (optimization_df.Categorisation_OED == which)                
            ])
        rows.append(row)
    tab = pd.DataFrame(rows)
    tables[option] = tab

sum_table = tables[HYBRID].applymap(lambda v: f"{v}").add(tables[NONHYBRID].applymap(lambda v: f" / {v}")).add(tables[FIXED].applymap(lambda v: f" / {v}"))
sum_table["problem"] = sum_table.problem.apply(lambda v: v.split("/")[0].strip())
sum_table["Qubits"] = sum_table.qubits.apply(lambda v: v.split("/")[0].strip() + " qubits")
sum_table["Arbitrary"] = sum_table.Arbitrary.apply(lambda v: v.split("/")[0].strip())

sum_table["Problem"] = "[" + sum_table["Arbitrary"] + "]" + sum_table["problem"]
sum_table = sum_table.sort_values(by=["Qubits", "problem"], key=lambda col: col.str.lower())
sum_table.index = pd.MultiIndex.from_frame(sum_table[["Qubits", "Problem"]])
sum_table = sum_table.drop(columns=["problem", "qubits", "Arbitrary", "Problem", "Qubits"])

RQ22_table = sum_table
RQ22_table

Hybrid
NonHybrid
Fixed


Optimized  Pareto Equal   
Qubits   Problem                                                      
2 qubits [True]AA2                       30 / 30 / 30     0 / 0 / 0  \
         [False]GHZ2                        0 / 0 / 0  30 / 30 / 30   
         [True]hamiltonian_simulation_2  30 / 30 / 30     0 / 0 / 0   
         [True]iswap_n2                  30 / 30 / 30     0 / 0 / 0   
         [True]QFT2                       12 / 28 / 0     7 / 0 / 0   
         [True]quantum_walk              30 / 30 / 30     0 / 0 / 0   
         [False]wstate2                  30 / 30 / 30     0 / 0 / 0   
3 qubits [True]AA3                         27 / 0 / 0     0 / 0 / 0   
         [True]fredkin_n3                   9 / 0 / 0     0 / 0 / 0   
         [False]GHZ3                     30 / 25 / 11     0 / 5 / 1   
         [False]GS3                       28 / 21 / 0     1 / 4 / 1   
         [True]hamiltonian_simulation_3   30 / 18 / 0     0 / 0 / 0   
         [False]linearsolver_n3          30 / 30 / 30     0 / 0 / 0   
         [True]QFT3                         2 / 0 / 0     2 / 0 / 0   
         [True]quantum_mc_F              30 / 30 / 30     0 / 0 / 0   
         [True]teleportation_n3            19 / 3 / 0     0 / 0 / 0   
         [True]tofolli_n3                  24 / 0 / 0     0 / 0 / 0   
         [False]wstate3                   30 / 18 / 0     0 / 0 / 0   
         [False]wstate_n3                30 / 30 / 11     0 / 0 / 0   
4 qubits [True]AA4                       30 / 30 / 30     0 / 0 / 0   
         [True]adder_n4                     9 / 0 / 0     0 / 0 / 0   
         [False]bell_n4                   30 / 26 / 0     0 / 0 / 0   
         [False]cat_state_n4                5 / 7 / 1     2 / 0 / 0   
         [False]GHZ4                        5 / 7 / 1     2 / 0 / 0   
         [False]GS4                        12 / 4 / 0     5 / 1 / 0   
         [True]hamiltonian_simulation_4    24 / 0 / 0     0 / 0 / 0   
         [True]hs4_n4                      25 / 5 / 0     0 / 0 / 0   
         [True]QFT4                         0 / 0 / 0     0 / 0 / 0   
         [False]qrng_n4                     0 / 0 / 0  27 / 30 / 30   
         [False]wstate4                    28 / 5 / 0     0 / 0 / 0   
5 qubits [True]AA5                       30 / 30 / 30     0 / 0 / 0   
         [False]GHZ5                        0 / 0 / 0     0 / 0 / 0   
         [False]GS5                         2 / 0 / 0     0 / 0 / 0   
         [True]hamiltonian_simulation_5    15 / 0 / 0     1 / 0 / 0   
         [False]lpn_n5                    23 / 26 / 2     1 / 0 / 0   
         [True]qec_en_n5                    0 / 0 / 0     0 / 0 / 0   
         [True]QFT5                         0 / 0 / 0     0 / 0 / 0   
         [False]wstate5                    12 / 0 / 0     0 / 0 / 0   

                                             Worse        Faulty  
Qubits   Problem                                                  
2 qubits [True]AA2                       0 / 0 / 0     0 / 0 / 0  
         [False]GHZ2                     0 / 0 / 0     0 / 0 / 0  
         [True]hamiltonian_simulation_2  0 / 0 / 0     0 / 0 / 0  
         [True]iswap_n2                  0 / 0 / 0     0 / 0 / 0  
         [True]QFT2                      0 / 0 / 0   11 / 2 / 30  
         [True]quantum_walk              0 / 0 / 0     0 / 0 / 0  
         [False]wstate2                  0 / 0 / 0     0 / 0 / 0  
3 qubits [True]AA3                       0 / 0 / 0   3 / 30 / 30  
         [True]fredkin_n3                0 / 0 / 0  21 / 30 / 30  
         [False]GHZ3                     0 / 0 / 0    0 / 0 / 18  
         [False]GS3                      1 / 0 / 0    0 / 5 / 29  
         [True]hamiltonian_simulation_3  0 / 0 / 0   0 / 12 / 30  
         [False]linearsolver_n3          0 / 0 / 0     0 / 0 / 0  
         [True]QFT3                      0 / 0 / 0  26 / 30 / 30  
         [True]quantum_mc_F              0 / 0 / 0     0 / 0 / 0  
         [True]teleportation_n3          0 / 0 / 0  11 / 27 / 30

## Create Summary Table (because the full table is too long!)

In [22]:
tables = {}
for option in [HYBRID, NONHYBRID, FIXED]:
    print(better_options[option])
    rows = []
    for problem in optimization_df.problem.unique():
        # print(problem)
        qubits=optimization_df[optimization_df.problem == problem].qubits.iloc[0]
        arbitrary = optimization_df[optimization_df.problem == problem].arbitrary.iloc[0]
        if arbitrary:
            arb = "Arbitrary"
        else:
            arb = "Specific"
        row = dict(problem = problem, qubits=qubits, Arbitrary=arb)

        for which in ["Optimized", "Pareto Equal", "Worse", "Faulty"]:
            row[which] = len(optimization_df[
                (optimization_df.option == option) & 
                (optimization_df.problem == problem) & 
                (optimization_df.Categorisation_OED == which)                
            ])
        rows.append(row)
    tab = pd.DataFrame(rows)
    tables[option] = tab
    tab = tab.sum().drop(columns=["problem", "qubits"])
    tables[option] = pd.DataFrame(tab)
        
sum_table = tables[HYBRID].applymap(lambda v: f"{v}").add(tables[NONHYBRID].applymap(lambda v: f" / {v}")).add(tables[FIXED].applymap(lambda v: f" / {v}"))

RQ22_summary_table_sum = sum_table.T.drop(columns=["qubits", "problem", "Arbitrary"])
RQ22_summary_table_sum

Hybrid
NonHybrid
Fixed


,Optimized,Pareto Equal,Worse,Faulty
0,701 / 493 / 296,78 / 70 / 62,2 / 1 / 0,359 / 576 / 782


In [23]:
tables = {}
for option in [HYBRID, NONHYBRID, FIXED]:
    print(better_options[option])
    rows = []
    for problem in optimization_df.problem.unique():
        # print(problem)
        qubits=optimization_df[optimization_df.problem == problem].qubits.iloc[0]
        arbitrary = optimization_df[optimization_df.problem == problem].arbitrary.iloc[0]
        if arbitrary:
            arb = "Arbitrary"
        else:
            arb = "Specific"
        row = dict(problem = problem, qubits=qubits, Arbitrary=arb)

        for which in ["Optimized", "Pareto Equal", "Worse", "Faulty"]:
            row[which] = len(optimization_df[
                (optimization_df.option == option) & 
                (optimization_df.problem == problem) & 
                (optimization_df.Categorisation_OED == which)                
            ])
        rows.append(row)
    tab = pd.DataFrame(rows)  
    tab = tab.groupby("Arbitrary").sum().drop(columns=["problem", "qubits"])
    tables[option] = tab

sum_table = tables[HYBRID].applymap(lambda v: f"{v}").add(tables[NONHYBRID].applymap(lambda v: f" / {v}")).add(tables[FIXED].applymap(lambda v: f" / {v}"))

RQ22_summary_table_input_state = sum_table
RQ22_summary_table_input_state

Hybrid
NonHybrid
Fixed


,Optimized,Pareto Equal,Worse,Faulty
Arbitrary,,,,
Arbitrary,406 / 264 / 210,10 / 0 / 0,0 / 0 / 0,214 / 366 / 420
Specific,295 / 229 / 86,68 / 70 / 62,2 / 1 / 0,145 / 210 / 362


In [24]:
tables = {}
for option in [HYBRID, NONHYBRID, FIXED]:
    print(better_options[option])
    rows = []
    for problem in optimization_df.problem.unique():
        # print(problem)
        qubits=optimization_df[optimization_df.problem == problem].qubits.iloc[0]
        arbitrary = optimization_df[optimization_df.problem == problem].arbitrary.iloc[0]
        if arbitrary:
            arb = "Arbitrary"
        else:
            arb = "Specific"
        row = dict(problem = problem, qubits=qubits, Arbitrary=arb)

        for which in ["Optimized", "Pareto Equal", "Worse", "Faulty"]:
            row[which] = len(optimization_df[
                (optimization_df.option == option) & 
                (optimization_df.problem == problem) & 
                (optimization_df.Categorisation_OED == which)                
            ])
        rows.append(row)
    tab = pd.DataFrame(rows)
    tab = tab.groupby("qubits").sum().drop(columns=["problem", "Arbitrary"])
    tables[option] = tab
    
sum_table = tables[HYBRID].applymap(lambda v: f"{v}").add(tables[NONHYBRID].applymap(lambda v: f" / {v}")).add(tables[FIXED].applymap(lambda v: f" / {v}"))

RQ22_summary_table_qubits = sum_table
RQ22_summary_table_qubits

Hybrid
NonHybrid
Fixed


,Optimized,Pareto Equal,Worse,Faulty
qubits,,,,
2,162 / 178 / 150,37 / 30 / 30,0 / 0 / 0,11 / 2 / 30
3,289 / 175 / 82,3 / 9 / 2,1 / 0 / 0,67 / 176 / 276
4,168 / 84 / 32,36 / 31 / 30,1 / 1 / 0,125 / 214 / 268
5,82 / 56 / 32,2 / 0 / 0,0 / 0 / 0,156 / 184 / 208


# Merge Tables for RQ1 and RQ2 and produce tex output

In [25]:
def tex_adapt_table(table, col_fix=True, space_slash=True):
    tex = table.to_latex()
    
    # centre columns
    if re.search(r"\{l+\}", tex):
        orig_cols = re.search(r"\{l+\}", tex).group(0)
        cols = orig_cols.replace("l", "c")
        tex = tex.replace(orig_cols, cols)
    
    if col_fix:
        tex = tex.replace("multicolumn{4}{r}{", "multicolumn{4}{c}{")
        tex = tex.replace("multicolumn{7}{r}{", "multicolumn{7}{c}{")
        tex = tex.replace("Qubits & Problem &  &  &  &  &  &  &  &  \\\\", "")
        
        tex = tex.replace(" &  & Optimized & Pareto Equal & Worse & Faulty & Optimized & Pareto Equal & Worse & Faulty",
                          "Qubits & Problem & Optimized & Pareto Equal & Worse & Faulty & Optimized & Pareto Equal & Worse & Faulty")
        
        if "ccccccccccccccc" in tex:
            tex = tex.replace("ccccccccccccccc", "c|ccccccc|ccccccc")
        elif "cccccccccc" in tex:
            tex = tex.replace("cccccccccc", "cc|cccc|cccc")
        elif "lrrrrrrrrrrrrrrr" in tex:
            tex = tex.replace("lrrrrrrrrrrrrrrr", "c|rrrrr|rrrrr|rrrrr")
        elif "ccccccccc" in tex:
            tex = tex.replace("ccccccccc", "ccccc|cccc")
            
    # vertically center multirow label
    tex = tex.replace("\\multirow[t]{3}{*}", "\\multirow[t]{3}{*}[-1em]")
    tex = tex.replace("PI", "PI Comparison")
    
    # centre column group headlines
    tex = tex.replace("{l}", "{c}")
    
    tex = tex.replace(f"Fixed", "\\fix")
    tex = tex.replace(f"Non-Hybrid", "\\non")
    tex = tex.replace(f"Hybrid", "\\hyb")
    
    tex = tex.replace("\\hyb_{NGen=50}", "NGen=50")
    tex = tex.replace("\\hyb_{NGen=100}", "NGen=100")
    tex = tex.replace("\\hyb_{N=100}", "N=100")
    tex = tex.replace("\\hyb_{N=200}", "N=200")
    tex = tex.replace("\\hyb_{Init=20}", "Init=20")
    tex = tex.replace("\\hyb_{Q2}", "Q2")

    
    tex = tex.replace("\\cline{1-5}\n\\bottomrule", "\\bottomrule")
    tex = tex.replace("\\cline{1-10}\n\\bottomrule", "\\bottomrule")
    
    
    # problem names
    tex = tex.replace('hamiltonian_simulation_', 'hamiltonian\_').replace('iswap_n2','iswap\_n2')
    tex = tex.replace('quantum_walk', 'quantum\_walk')
    tex = tex.replace('fredkin_n3', 'fredkin\_n3')
    tex = tex.replace('linearsolver_n3',  'linearsolver\_n3')
    tex = tex.replace('quantum_mc_F', 'quantum\_mc\_F')
    tex = tex.replace('teleportation_n3',  'teleportation\_n3')
    tex = tex.replace('tofolli_n3', 'tofolli\_n3')
    tex = tex.replace('wstate_n3',  'wstate\_n3')
    tex = tex.replace('adder_n4',  'adder\_n4')
    tex = tex.replace('bell_n4', 'bell\_n4')
    tex = tex.replace('cat_state_n4', 'cat\_state\_n4')
    tex = tex.replace('hs4_n4',  'hs4\_n4')
    tex = tex.replace('qrng_n4', 'qrng\_n4')
    tex = tex.replace('lpn_n5',  'lpn\_n5')
    tex = tex.replace('qec_en_n5', 'qec\_en\_n5')
    
    
    tex = tex.replace('QSE_', 'QSE\_')
    tex = tex.replace('QSE2_', 'QSE2\_')
    tex = tex.replace('QSO_', 'QSO\_')
    tex = tex.replace('QG_', 'QG\_')
    
    # replace rowcolor
    
    colored_lines = []
    for line in tex.split("\\\\"):
        # print("before", line)
        if '[True]' in line:  # make line arbitrary
            line = line.replace('&', '& \\cellcolor{specificrow}')
            line = line.replace("[True]", "")
        elif '[False]' in line:  # make line specific
            line = line.replace('&', '& \\cellcolor{arbitraryrow}')
            line = line.replace("[False]", "")
        else:
            line = line    
        colored_lines.append(line)
    
    tex = "\\\\".join(colored_lines)
    
    # RQ1 Table:
    # add lines between new Qubit sizes
    tex = tex.replace("cline{1-10}", "hline")
    
    # RQ2 Summary Table:
    tex = tex.replace("Repair &", "\\hline Repair &")
    tex = tex.replace("Specific &", "\\hline Specific &")
    tex = tex.replace("2 qubits &", "\\hline 2 qubits &")
    
    # Don't make spaces so large
    if space_slash:
        tex = tex.replace(" / ", "{\,}/{\,}")
    else:
        tex = tex.replace(" / ", "/")
    
    return tex

def write_table_to_file(table, filepath, col_fix=True, space_slash=True):
    tex = tex_adapt_table(table, col_fix=col_fix, space_slash=space_slash)
    with open(filepath, 'w') as texfile:
        texfile.write(tex)
    return tex

## RQ1

In [26]:
RQ11_table_out = RQ11_table.copy()
RQ12_table_out = RQ12_table.copy()

RQ11_table_out.columns = pd.MultiIndex.from_product([["RQ1.1 (Perfect Accuracy)"], RQ11_table_out.columns])
RQ12_table_out.columns = pd.MultiIndex.from_product([["RQ1.2 (Acceptable Accuracy)"], RQ12_table_out.columns])

RQ1_table_out = pd.concat([RQ11_table_out, RQ12_table_out], axis=1)
display(RQ1_table_out)

write_table_to_file(RQ1_table_out, paper_tables_dir / f"{DATA_PREFIX}-RQ1.tex", col_fix=True )

RQ1.1 (Perfect Accuracy)                
                                             Optimized Pareto Equal   
Input State Problem                                                   
Specific    QG_8 (2 qubits)               30 / 30 / 30    0 / 0 / 0  \
            QSO_6 (2 qubits)                26 / 0 / 0  4 / 30 / 30   
            QSO_5 (3 qubits)              30 / 30 / 30    0 / 0 / 0   
            QSE_15 (4 qubits)             30 / 30 / 30    0 / 0 / 0   
            QSE_3 (5 qubits)              30 / 30 / 30    0 / 0 / 0   
Arbitrary   QSE2_2 (2 qubits)               30 / 0 / 0   0 / 0 / 30   
            QSE2_3 (3 qubits)                0 / 0 / 0   17 / 0 / 0   
            QSE2_4 (4 qubits)                0 / 0 / 0    0 / 0 / 0   
            QSE2_5 (5 qubits)                0 / 0 / 0    0 / 0 / 0   

                                                          
                                    Worse        Faulty   
Input State Problem                                       
Specific    QG_8 (2 qubits)     0 / 0 / 0     0 / 0 / 0  \
            QSO_6 (2 qubits)    0 / 0 / 0     0 / 0 / 0   
            QSO_5 (3 qubits)    0 / 0 / 0     0 / 0 / 0   
            QSE_15 (4 qubits)   0 / 0 / 0     0 / 0 / 0   
            QSE_3 (5 qubits)    0 / 0 / 0     0 / 0 / 0   
Arbitrary   QSE2_2 (2 qubits)  0 / 16 / 0    0 / 14 / 0   
            QSE2_3 (3 qubits)   8 / 0 / 8   5 / 30 / 22   
            QSE2_4 (4 qubits)   7 / 0 / 0  23 / 30 / 30   
            QSE2_5 (5 qubits)   6 / 0 / 0  24 / 30 / 30   

                              RQ1.2 (Acceptable Accuracy)                
                                                Optimized Pareto Equal   
Input State Problem                                                      
Specific    QG_8 (2 qubits)                  30 / 30 / 30    0 / 0 / 0  \
            QSO_6 (2 qubits)                   26 / 0 / 0  4 / 30 / 30   
            QSO_5 (3 qubits)                 30 / 30 / 30    0 / 0 / 0   
            QSE_15 (4 qubits)                30 / 30 / 30    0 / 0 / 0   
            QSE_3 (5 qubits)                 30 / 30 / 30    0 / 0 / 0   
Arbitrary   QSE2_2 (2 qubits)                  30 / 0 / 0   0 / 0 / 30   
            QSE2_3 (3 qubits)                  14 / 0 / 0   14 / 0 / 0   
            QSE2_4 (4 qubits)                   1 / 0 / 0    3 / 0 / 0   
            QSE2_5 (5 qubits)                   0 / 0 / 0    6 / 0 / 0   

                                                        
                                   Worse        Faulty  
Input State Problem                                     
Specific    QG_8 (2 qubits)    0 / 0 / 0     0 / 0 / 0  
            QSO_6 (2 qubits)   0 / 0 / 0     0 / 0 / 0  
            QSO_5 (3 qubits)   0 / 0 / 0     0 / 0 / 0  
            QSE_15 (4 qubits)  0 / 0 / 0     0 / 0 / 0  
            QSE_3 (5 qubits)   0 / 0 / 0     0 / 0 / 0  
Arbitrary   QSE2_2 (2 qubits)  0 / 0 / 0    0 / 30 / 0  
            QSE2_3 (3 qubits)  0 / 0 / 0   2 / 30 / 30  
            QSE2_4 (4 qubits)  1 / 0 / 0  25 / 30 / 30  
            QSE2_5 (5 qubits)  0 / 0 / 0  24 / 30 / 30

'\\begin{tabular}{cc|cccc|cccc}\n\\toprule\n &  & \\multicolumn{4}{c}{RQ1.1 (Perfect Accuracy)} & \\multicolumn{4}{c}{RQ1.2 (Acceptable Accuracy)} \\\\\nQubits & Problem & Optimized & Pareto Equal & Worse & Faulty & Optimized & Pareto Equal & Worse & Faulty \\\\\nInput State & Problem &  &  &  &  &  &  &  &  \\\\\n\\midrule\n\\multirow[t]{5}{*}{Specific} & QG\\_8 (2 qubits) & 30{\\,}/{\\,}30{\\,}/{\\,}30 & 0{\\,}/{\\,}0{\\,}/{\\,}0 & 0{\\,}/{\\,}0{\\,}/{\\,}0 & 0{\\,}/{\\,}0{\\,}/{\\,}0 & 30{\\,}/{\\,}30{\\,}/{\\,}30 & 0{\\,}/{\\,}0{\\,}/{\\,}0 & 0{\\,}/{\\,}0{\\,}/{\\,}0 & 0{\\,}/{\\,}0{\\,}/{\\,}0 \\\\\n & QSO\\_6 (2 qubits) & 26{\\,}/{\\,}0{\\,}/{\\,}0 & 4{\\,}/{\\,}30{\\,}/{\\,}30 & 0{\\,}/{\\,}0{\\,}/{\\,}0 & 0{\\,}/{\\,}0{\\,}/{\\,}0 & 26{\\,}/{\\,}0{\\,}/{\\,}0 & 4{\\,}/{\\,}30{\\,}/{\\,}30 & 0{\\,}/{\\,}0{\\,}/{\\,}0 & 0{\\,}/{\\,}0{\\,}/{\\,}0 \\\\\n & QSO\\_5 (3 qubits) & 30{\\,}/{\\,}30{\\,}/{\\,}30 & 0{\\,}/{\\,}0{\\,}/{\\,}0 & 0{\\,}/{\\,}0{\\,}/{\\,}0 & 0{\\,}/{\\,}0{\\,}

## RQ2 -- Summary Tables

In [27]:
RQ21_summary_table_sum_out = RQ21_summary_table_sum.copy()
RQ21_summary_table_input_state_out = RQ21_summary_table_input_state.copy()
RQ21_summary_table_qubits_out = RQ21_summary_table_qubits.copy()

RQ22_summary_table_sum_out = RQ22_summary_table_sum.copy()
RQ22_summary_table_input_state_out = RQ22_summary_table_input_state.copy()
RQ22_summary_table_qubits_out = RQ22_summary_table_qubits.copy()

RQ21_summary_concat = pd.concat([RQ21_summary_table_sum_out, RQ21_summary_table_input_state_out, RQ21_summary_table_qubits_out])
RQ21_summary_concat.columns = pd.MultiIndex.from_product([["RQ2.1 (Perfect Accuracy)"], RQ21_summary_concat.columns])

RQ22_summary_concat = pd.concat([RQ22_summary_table_sum_out, RQ22_summary_table_input_state_out, RQ22_summary_table_qubits_out])
RQ22_summary_concat.columns = pd.MultiIndex.from_product([["RQ2.2 (Acceptable Accuracy)"], RQ22_summary_concat.columns])

RQ2_summary_out = pd.concat([RQ21_summary_concat, RQ22_summary_concat], axis=1)
RQ2_summary_out.index = ["Total", "Arbitrary", "Specific", "2 qubits", "3 qubits", "4 qubits", "5 qubits"]
RQ2_summary_out = RQ2_summary_out.reindex(["Total", "Specific", "Arbitrary", "2 qubits", "3 qubits", "4 qubits", "5 qubits"])

display(RQ2_summary_out)

write_table_to_file(RQ2_summary_out, paper_tables_dir / f"{DATA_PREFIX}-RQ2-summary.tex", col_fix=True )

RQ2.1 (Perfect Accuracy)                                
                         Optimized   Pareto Equal         Worse   
Total              541 / 135 / 105  143 / 84 / 64  50 / 10 / 14  \
Specific             274 / 63 / 45   89 / 84 / 64  23 / 10 / 14   
Arbitrary            267 / 72 / 60     54 / 0 / 0    27 / 0 / 0   
2 qubits            135 / 101 / 90   40 / 30 / 30    19 / 0 / 0   
3 qubits             248 / 17 / 11    37 / 20 / 2    9 / 7 / 14   
4 qubits               120 / 6 / 2   55 / 32 / 30    14 / 3 / 0   
5 qubits               38 / 11 / 2     11 / 2 / 2     8 / 0 / 0   

                           RQ2.2 (Acceptable Accuracy)                 
                    Faulty                   Optimized  Pareto Equal   
Total      406 / 911 / 957             701 / 493 / 296  78 / 70 / 62  \
Specific   124 / 353 / 387              295 / 229 / 86  68 / 70 / 62   
Arbitrary  282 / 558 / 570             406 / 264 / 210    10 / 0 / 0   
2 qubits      16 / 79 / 90             162 / 178 / 150  37 / 30 / 30   
3 qubits    66 / 316 / 333              289 / 175 / 82     3 / 9 / 2   
4 qubits   141 / 289 / 298               168 / 84 / 32  36 / 31 / 30   
5 qubits   183 / 227 / 236                82 / 56 / 32     2 / 0 / 0   

                                       
               Worse           Faulty  
Total      2 / 1 / 0  359 / 576 / 782  
Specific   2 / 1 / 0  145 / 210 / 362  
Arbitrary  0 / 0 / 0  214 / 366 / 420  
2 qubits   0 / 0 / 0      11 / 2 / 30  
3 qubits   1 / 0 / 0   67 / 176 / 276  
4 qubits   1 / 1 / 0  125 / 214 / 268  
5 qubits   0 / 0 / 0  156 / 184 / 208

'\\begin{tabular}{ccccc|cccc}\n\\toprule\n & \\multicolumn{4}{c}{RQ2.1 (Perfect Accuracy)} & \\multicolumn{4}{c}{RQ2.2 (Acceptable Accuracy)} \\\\\n & Optimized & Pareto Equal & Worse & Faulty & Optimized & Pareto Equal & Worse & Faulty \\\\\n\\midrule\nTotal & 541{\\,}/{\\,}135{\\,}/{\\,}105 & 143{\\,}/{\\,}84{\\,}/{\\,}64 & 50{\\,}/{\\,}10{\\,}/{\\,}14 & 406{\\,}/{\\,}911{\\,}/{\\,}957 & 701{\\,}/{\\,}493{\\,}/{\\,}296 & 78{\\,}/{\\,}70{\\,}/{\\,}62 & 2{\\,}/{\\,}1{\\,}/{\\,}0 & 359{\\,}/{\\,}576{\\,}/{\\,}782 \\\\\n\\hline Specific & 274{\\,}/{\\,}63{\\,}/{\\,}45 & 89{\\,}/{\\,}84{\\,}/{\\,}64 & 23{\\,}/{\\,}10{\\,}/{\\,}14 & 124{\\,}/{\\,}353{\\,}/{\\,}387 & 295{\\,}/{\\,}229{\\,}/{\\,}86 & 68{\\,}/{\\,}70{\\,}/{\\,}62 & 2{\\,}/{\\,}1{\\,}/{\\,}0 & 145{\\,}/{\\,}210{\\,}/{\\,}362 \\\\\nArbitrary & 267{\\,}/{\\,}72{\\,}/{\\,}60 & 54{\\,}/{\\,}0{\\,}/{\\,}0 & 27{\\,}/{\\,}0{\\,}/{\\,}0 & 282{\\,}/{\\,}558{\\,}/{\\,}570 & 406{\\,}/{\\,}264{\\,}/{\\,}210 & 10{\\,}/{\\,}0{\\,}/{\\,}0 & 

## RQ2 -- Detail Table

In [28]:
RQ21_table_out = RQ21_table.copy()
RQ22_table_out = RQ22_table.copy()
RQ21_table_out.columns = pd.MultiIndex.from_product([["RQ2.1 (Perfect Accuracy)"], RQ21_table_out.columns])
RQ22_table_out.columns = pd.MultiIndex.from_product([["RQ2.2 (Acceptable Accuracy)"], RQ22_table_out.columns])

RQ2_table_out = pd.concat([RQ21_table_out, RQ22_table_out], axis=1)
display(RQ2_table_out)

write_table_to_file(RQ2_table_out, paper_tables_dir / f"{DATA_PREFIX}-RQ2.tex", col_fix=True )

RQ2.1 (Perfect Accuracy)   
                                                       Optimized   
Qubits   Problem                                                   
2 qubits [True]AA2                                   30 / 6 / 30  \
         [False]GHZ2                                   0 / 0 / 0   
         [True]hamiltonian_simulation_2               30 / 7 / 0   
         [True]iswap_n2                             30 / 30 / 30   
         [True]QFT2                                  12 / 28 / 0   
         [True]quantum_walk                            3 / 0 / 0   
         [False]wstate2                             30 / 30 / 30   
3 qubits [True]AA3                                    15 / 0 / 0   
         [True]fredkin_n3                              6 / 0 / 0   
         [False]GHZ3                                 30 / 6 / 11   
         [False]GS3                                   28 / 5 / 0   
         [True]hamiltonian_simulation_3               30 / 0 / 0   
         [False]linearsolver_n3                       30 / 6 / 0   
         [True]QFT3                                    0 / 0 / 0   
         [True]quantum_mc_F                           13 / 0 / 0   
         [True]teleportation_n3                       18 / 0 / 0   
         [True]tofolli_n3                             18 / 0 / 0   
         [False]wstate3                               30 / 0 / 0   
         [False]wstate_n3                             30 / 0 / 0   
4 qubits [True]AA4                                     0 / 0 / 0   
         [True]adder_n4                                3 / 0 / 0   
         [False]bell_n4                               30 / 0 / 0   
         [False]cat_state_n4                           5 / 2 / 1   
         [False]GHZ4                                   5 / 2 / 1   
         [False]GS4                                   13 / 1 / 0   
         [True]hamiltonian_simulation_4               23 / 0 / 0   
         [True]hs4_n4                                 24 / 1 / 0   
         [True]QFT4                                    0 / 0 / 0   
         [False]qrng_n4                                0 / 0 / 0   
         [False]wstate4                               17 / 0 / 0   
5 qubits [True]AA5                                     0 / 0 / 0   
         [False]GHZ5                                   0 / 0 / 0   
         [False]GS5                                    2 / 0 / 0   
         [True]hamiltonian_simulation_5               12 / 0 / 0   
         [False]lpn_n5                               23 / 11 / 2   
         [True]qec_en_n5                               0 / 0 / 0   
         [True]QFT5                                    0 / 0 / 0   
         [False]wstate5                                1 / 0 / 0   

                                                                    
                                         Pareto Equal       Worse   
Qubits   Problem                                                    
2 qubits [True]AA2                          0 / 0 / 0   0 / 0 / 0  \
         [False]GHZ2                     30 / 30 / 30   0 / 0 / 0   
         [True]hamiltonian_simulation_2     0 / 0 / 0   0 / 0 / 0   
         [True]iswap_n2                     0 / 0 / 0   0 / 0 / 0   
         [True]QFT2                         7 / 0 / 0  10 / 0 / 0   
         [True]quantum_walk                 3 / 0 / 0   9 / 0 / 0   
         [False]wstate2                     0 / 0 / 0   0 / 0 / 0   
3 qubits [True]AA3                         11 / 0 / 0   0 / 0 / 0   
         [True]fredkin_n3                   7 / 0 / 0   5 / 0 / 0   
         [False]GHZ3                       0 / 13 / 1  0 / 6 / 13   
         [False]GS3                         1 / 7 / 1   1 / 1 / 1   
         [True]hamiltonian_simulation_3     0 / 0 / 0   0 / 0 / 0   
         [False]linearsolver_n3             0 / 0 / 0   0 / 0 / 0   
         [True]QFT3                         0 / 0 / 0   2 / 0 / 0   
         [True]quantum_mc_F                 2 / 0 / 0   1 / 0 / 0   
         

'\\begin{tabular}{cc|cccc|cccc}\n\\toprule\n &  & \\multicolumn{4}{c}{RQ2.1 (Perfect Accuracy)} & \\multicolumn{4}{c}{RQ2.2 (Acceptable Accuracy)} \\\\\nQubits & Problem & Optimized & Pareto Equal & Worse & Faulty & Optimized & Pareto Equal & Worse & Faulty \\\\\n\n\\midrule\n\\multirow[t]{7}{*}{2 qubits} & \\cellcolor{specificrow} AA2 & \\cellcolor{specificrow} 30{\\,}/{\\,}6{\\,}/{\\,}30 & \\cellcolor{specificrow} 0{\\,}/{\\,}0{\\,}/{\\,}0 & \\cellcolor{specificrow} 0{\\,}/{\\,}0{\\,}/{\\,}0 & \\cellcolor{specificrow} 0{\\,}/{\\,}24{\\,}/{\\,}0 & \\cellcolor{specificrow} 30{\\,}/{\\,}30{\\,}/{\\,}30 & \\cellcolor{specificrow} 0{\\,}/{\\,}0{\\,}/{\\,}0 & \\cellcolor{specificrow} 0{\\,}/{\\,}0{\\,}/{\\,}0 & \\cellcolor{specificrow} 0{\\,}/{\\,}0{\\,}/{\\,}0 \\\\\n & \\cellcolor{arbitraryrow} GHZ2 & \\cellcolor{arbitraryrow} 0{\\,}/{\\,}0{\\,}/{\\,}0 & \\cellcolor{arbitraryrow} 30{\\,}/{\\,}30{\\,}/{\\,}30 & \\cellcolor{arbitraryrow} 0{\\,}/{\\,}0{\\,}/{\\,}0 & \\cellcolor{arbitraryrow}

## RQ2 Calculate Relative Improvement (for those that optimized)

In [29]:
def extract_relative_optimization(row): 
    ref_num_gates, ref_depth, ref_non_local = reference_fitness_values[row.problem]
        
    rel_opt_num_gates = (ref_num_gates - row.num_gates) / ref_num_gates
    rel_depth = (ref_depth - row.depth) / ref_depth
    rel_non_local = (ref_non_local - row.num_nonloc_gates) / ref_non_local
    
    return rel_opt_num_gates, rel_depth, rel_non_local
    

### OL

In [30]:
all_vals_df = pd.read_pickle(OL_file_path)
all_vals_df["repair"] = all_vals_df.problem.apply(lambda p: p in repair_circuits)

In [31]:
optimization_df = all_vals_df[all_vals_df.repair == False].copy()
optimization_df["Categorisation_OL"] = optimization_df.apply(get_operator_categorisation_OL, axis=1)

In [32]:
optimized_df = optimization_df[optimization_df.Categorisation_OL == "Optimized"].copy()
optimized_df[["RelOptGates", "RelOptDepth", "RelOptNonLocal"]] = optimized_df.apply(extract_relative_optimization, axis=1, result_type="expand")

In [33]:
avg_opt_problem = optimized_df[optimized_df.option.isin([HYBRID, NONHYBRID, FIXED])].groupby(["option", "problem"])[["RelOptGates", "RelOptDepth", "RelOptNonLocal"]].mean().reset_index()

avg_optimization = {}
for option in [HYBRID, NONHYBRID, FIXED]:
    missing = 38 - len(avg_opt_problem[avg_opt_problem.option == option])
    avg_optimization[better_options[option]] = np.concatenate([avg_opt_problem[avg_opt_problem.option == option][["RelOptGates", "RelOptDepth", "RelOptNonLocal"]].to_numpy(),  np.zeros((missing, 3))]).mean(axis=0)

# Q2 Optimization
rows = []
for name, opt_vals in QISKIT_opt_results.items():
    if ((np.array(reference_fitness_values[name]) - np.array(opt_vals)) > 0 ).any():  # if any is better, calculate improvement
        rows.append(extract_relative_optimization(pd.Series(dict(problem=name, num_gates=opt_vals[0], depth=opt_vals[1], num_nonloc_gates=opt_vals[2]))))
    else:  # otherwise, use zeros
        rows.append([0,0,0])
        
avg_optimization["Q2"] = np.array(rows).mean(axis=0)
    
print("OL (Theoretical) in percent")
(pd.DataFrame(avg_optimization, index=["RelOptGates", "RelOptDepth", "RelOptNonLocal"]).T * 100).round(2)

OL (Theoretical) in percent


,RelOptGates,RelOptDepth,RelOptNonLocal
Hybrid,37.01,33.99,18.05
NonHybrid,13.41,13.46,7.40
Fixed,9.12,8.58,2.63
Q2,4.72,3.41,1.32


### OED

In [34]:
all_vals_df = pd.read_pickle(OED_file_path)
all_vals_df["repair"] = all_vals_df.problem.apply(lambda p: p in repair_circuits)

In [35]:
optimization_df = all_vals_df[all_vals_df.repair == False].copy()
optimization_df["Categorisation_OED"] = optimization_df.apply(get_operator_categorisation_OED, axis=1)

In [36]:
optimized_df = optimization_df[optimization_df.Categorisation_OED == "Optimized"].copy()
optimized_df[["RelOptGates", "RelOptDepth", "RelOptNonLocal"]] = optimized_df.apply(extract_relative_optimization, axis=1, result_type="expand")

In [37]:
avg_opt_problem = optimized_df[optimized_df.option.isin([HYBRID, NONHYBRID, FIXED])].groupby(["option", "problem"])[["RelOptGates", "RelOptDepth", "RelOptNonLocal"]].mean().reset_index()

avg_optimization = {}
for option in [HYBRID, NONHYBRID, FIXED]:
    missing = 38 - len(avg_opt_problem[avg_opt_problem.option == option])
    avg_optimization[better_options[option]] = np.concatenate([avg_opt_problem[avg_opt_problem.option == option][["RelOptGates", "RelOptDepth", "RelOptNonLocal"]].to_numpy(),  np.zeros((missing, 3))]).mean(axis=0)

# Q2 Optimization
rows = []
for name, opt_vals in QISKIT_opt_results.items():
    if ((np.array(reference_fitness_values[name]) - np.array(opt_vals)) > 0 ).any():  # if any is better, calculate improvement
        rows.append(extract_relative_optimization(pd.Series(dict(problem=name, num_gates=opt_vals[0], depth=opt_vals[1], num_nonloc_gates=opt_vals[2]))))
    else:  # otherwise, use zeros
        rows.append([0,0,0])
        
avg_optimization["Q2"] = np.array(rows).mean(axis=0)
    
print("OED (Practical) in percent")
(pd.DataFrame(avg_optimization, index=["RelOptGates", "RelOptDepth", "RelOptNonLocal"]).T * 100).round(2)

OED (Practical) in percent


,RelOptGates,RelOptDepth,RelOptNonLocal
Hybrid,44.58,43.21,30.32
NonHybrid,32.54,31.78,25.07
Fixed,24.76,23.59,17.65
Q2,4.72,3.41,1.32


In [38]:
tables = {}
for option in [k for k in better_options.keys() if k not in [FIXED, NONHYBRID]]:
    print(better_options[option])
    rows = []
    for problem in optimization_df.problem.unique():
        # print(problem)
        qubits=optimization_df[optimization_df.problem == problem].qubits.iloc[0]
        arbitrary = optimization_df[optimization_df.problem == problem].arbitrary.iloc[0]
        if arbitrary:
            arb = "Arbitrary"
        else:
            arb = "Specific"
        row = dict(problem = problem, qubits=qubits, Arbitrary=arb)

        for which in ["Optimized", "Pareto Equal", "Worse", "Faulty"]:
            row[which] = len(optimization_df[
                (optimization_df.option == option) & 
                (optimization_df.problem == problem) & 
                (optimization_df.Categorisation_OED == which)                
            ])
        rows.append(row)
    tab = pd.DataFrame(rows)
    # display(tab)
    # tab = tab.reindex(tab["Arbitrary"])
    tables[option] = tab
    
    
    # tab = tab.sum().drop(columns=["problem", "qubits"])
    # tables[option] = pd.DataFrame(tab)
    
for option, tab in tables.items():
    # display(tab)
    tab[better_options[option]] = tab.apply(lambda row: "/".join(row[["Optimized", "Pareto Equal", "Worse", "Faulty"]].astype(str)), axis=1)
    tab = tab.drop(columns=["Optimized", "Pareto Equal", "Worse", "Faulty"])
    tab["Problem"] = "[" + tab["Arbitrary"] + "]" + tab["problem"]
    tab = tab.sort_values(["qubits", "problem"])
    tab.index = pd.MultiIndex.from_frame(tab[["qubits", "Problem"]])
    
    tab = tab.drop(columns=["qubits", "problem", "Arbitrary", "Problem"])
    # tab.columns = pd.MultiIndex.from_product([[better_options[option]], tab.columns])
    
    
    tables[option] = tab
    
    # display(tab)
    
RQ42_summary_table_sum = pd.concat(tables.values(), axis=1)
RQ42_summary_table_sum

Hybrid
Hybrid_{NGen=50}
Hybrid_{NGen=100}
Hybrid_{N=100}
Hybrid_{N=200}
Hybrid_{Init=20}
Hybrid_{Q2}


Hybrid Hybrid_{NGen=50}   
qubits Problem                                                           
2      [Arbitrary]AA2                        30/0/0/0          0/0/0/0  \
       [Specific]GHZ2                        0/30/0/0          0/0/0/0   
       [Arbitrary]QFT2                      12/7/0/11          0/0/0/0   
       [Arbitrary]hamiltonian_simulation_2   30/0/0/0          0/0/0/0   
       [Arbitrary]iswap_n2                   30/0/0/0          0/0/0/0   
       [Arbitrary]quantum_walk               30/0/0/0          0/0/0/0   
       [Specific]wstate2                     30/0/0/0          0/0/0/0   
3      [Arbitrary]AA3                        27/0/0/3          0/0/0/0   
       [Specific]GHZ3                        30/0/0/0          0/0/0/0   
       [Specific]GS3                         28/1/1/0          0/0/0/0   
       [Arbitrary]QFT3                       2/2/0/26          0/0/0/0   
       [Arbitrary]fredkin_n3                 9/0/0/21          0/0/0/0   
       [Arbitrary]hamiltonian_simulation_3   30/0/0/0          0/0/0/0   
       [Specific]linearsolver_n3             30/0/0/0          0/0/0/0   
       [Arbitrary]quantum_mc_F               30/0/0/0          0/0/0/0   
       [Arbitrary]teleportation_n3          19/0/0/11          0/0/0/0   
       [Arbitrary]tofolli_n3                 24/0/0/6          0/0/0/0   
       [Specific]wstate3                     30/0/0/0          0/0/0/0   
       [Specific]wstate_n3                   30/0/0/0          0/0/0/0   
4      [Arbitrary]AA4                        30/0/0/0          0/0/0/0   
       [Specific]GHZ4                        5/2/0/23          0/0/0/0   
       [Specific]GS4                        12/5/1/12          0/0/0/0   
       [Arbitrary]QFT4                       0/0/0/30          0/0/0/0   
       [Arbitrary]adder_n4                   9/0/0/21          0/0/0/0   
       [Specific]bell_n4                     30/0/0/0          0/0/0/0   
       [Specific]cat_state_n4                5/2/0/23          0/0/0/0   
       [Arbitrary]hamiltonian_simulation_4   24/0/0/6          0/0/0/0   
       [Arbitrary]hs4_n4                     25/0/0/5          0/0/0/0   
       [Specific]qrng_n4                     0/27/0/3          0/0/0/0   
       [Specific]wstate4                     28/0/0/2          0/0/0/0   
5      [Arbitrary]AA5                        30/0/0/0          0/0/0/0   
       [Specific]GHZ5                        0/0/0/30          0/0/0/0   
       [Specific]GS5                         2/0/0/28          0/0/0/0   
       [Arbitrary]QFT5                       0/0/0/30          0/0/0/0   
       [Arbitrary]hamiltonian_simulation_5  15/1/0/14          0/0/0/0   
       [Specific]lpn_n5                      23/1/0/6          0/0/0/0   
       [Arbitrary]qec_en_n5                  0/0/0/30          0/0/0/0   
       [Specific]wstate5                    12/0/0/18          0/0/0/0   

                                           Hybrid_{NGen=100} Hybrid_{N=100}   
qubits Problem                                                                
2      [Arbitrary]AA2                                0/0/0/0       30/0/0/0  \
       [Specific]GHZ2                                0/0/0/0       0/30/0/0   
       [Arbitrary]QFT2                               0/0/0/0       27/3/0/0   
       [Arbitrary]hamiltonian_simulation_2           0/0/0/0       30/0/0/0   
       [Arbitrary]iswap_n2                           0/0/0/0       30/0/0/0   
       [Arbitrary]quantum_walk                       0/0/0/0       30/0/0/0   
       [Specific]wstate2                             0/0/0/0       30/0/0/0   
3      [Arbitrary]AA3                                0/0/0/0       29/0/0/1   
       [Specific]GHZ3                                0/0/0/0       30/0/0/0   
       [Specific]GS3                                 0/0/0/0       30/0/0/0   
       [Arbitrary]QFT3                               0/0/0/0       5/1/0/24   
       [Arbitrary]fredkin_n3                         0/0/0/

# RQ4 

In [39]:
def extract_best_solutions_max_gen(files, earliest_finish=None, max_gen=100):
    earliest_finish_times = {}
    if earliest_finish:
        for earliest_finish_file in list(results_path.glob("*earliest_finish.csv")):
            problem = earliest_finish_file.stem.replace("_earliest_finish", "")

            earliest_finish_df = pd.read_csv(earliest_finish_file)
            earliest_finish_time = earliest_finish_df.iloc[0][earliest_finish]
            earliest_finish_times[problem] = earliest_finish_time
    # print("Earliest finish times:")
    # pprint(earliest_finish_times)
    best_OL_rows = []
    best_OED_rows = []
    # extract last gen value
    for pi_file in sorted(files):
        problem, seed, option = extract_info_from_file(pi_file)
        qubits, arbitrary = QUBITS_and_ARBITRARY[problem]
        
        last_row = dict(problem=problem, option=option, seed=seed, qubits=qubits, arbitrary=arbitrary)
        results_file_df = pd.read_csv(pi_file)
    
        if max_gen and max_gen > 0:
            results_file_df = results_file_df[results_file_df.ngen < max_gen]  # filter max_gen
            results_file_df["option"] = f"Ngen_{max_gen}"
        
        if earliest_finish:
            if problem not in earliest_finish_times:
                print("WARNING!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
                print("No earliest finish time for problem", problem)
            else:
                results_file_df = results_file_df[results_file_df.timestamp <= earliest_finish_df[earliest_finish].iloc[0]]
        
        last_gen_df = results_file_df[results_file_df.ngen == results_file_df.ngen.max()].reset_index()
        last_gen_df["no_error_prob_actual"] = 1 - last_gen_df.apply(lambda row: get_actual_error_rate(row.num_gates, row.num_nonloc_gates), axis=1)
        last_gen_df["OED"] = last_gen_df.overlap * last_gen_df.no_error_prob_actual
        
        best_OL_row = dict(last_row)
        best_OL_row.update(last_gen_df.sort_values("overlap", ascending=False).iloc[0].to_dict())
        best_OL_rows.append(best_OL_row)
        
        best_OED_row = dict(last_row)
        best_OED_row.update(last_gen_df.sort_values("OED", ascending=False).iloc[0].to_dict())
        best_OED_rows.append(best_OED_row)

    OL_df = pd.DataFrame(best_OL_rows).sort_values(by=["problem", "option", "seed"])
    OED_df = pd.DataFrame(best_OED_rows).sort_values(by=["problem", "option", "seed"])
    
    return OL_df, OED_df

In [40]:
output_files = list(raw_results_path.glob("*.csv"))
output_files = [f for f in output_files if "logbook" not in str(f)]
output_files = [f for f in output_files if "seed" in str(f)]
output_files = [f for f in output_files if "PI.csv" not in str(f)]
output_files = [f for f in output_files if "DCI.csv" not in str(f)]
output_files = [f for f in output_files if "HVrefpoint.csv" not in str(f)]
output_files = [f for f in output_files if "globalPareto.csv" not in str(f)]
output_files = [f for f in output_files if "earliest_finish.csv" not in str(f)]

hybrid_only_files = output_files
hybrid_only_files = [f for f in hybrid_only_files if "gateset_fixed" not in str(f)]
hybrid_only_files = [f for f in hybrid_only_files if "Q2" not in str(f)]
hybrid_only_files = [f for f in hybrid_only_files if "init_pop" not in str(f)]
hybrid_only_files = [f for f in hybrid_only_files if "N_100" not in str(f)]
hybrid_only_files = [f for f in hybrid_only_files if "N_200" not in str(f)]
len(hybrid_only_files)
# hybrid_only_files

1410

## Get the data for the Hybrid Search after 50 and 100 generations

In [41]:
%%time
OL_file_path_100gen = Path(f"{DATA_PREFIX.replace('%', '')}_vals_OL_100gen.pkl")
OL_file_path_50gen = Path(f"{DATA_PREFIX.replace('%', '')}_vals_OL_50gen.pkl")
OED_file_path_100gen = Path(f"{DATA_PREFIX.replace('%', '')}_vals_OED_100gen.pkl")
OED_file_path_50gen = Path(f"{DATA_PREFIX.replace('%', '')}_vals_OED_50gen.pkl")

ngen_100_OL_df = None
ngen_100_OED_df = None

if OL_file_path_100gen.exists() and OED_file_path_100gen.exists():
    ngen_100_OL_df = pd.read_pickle(OL_file_path_100gen)
    ngen_100_OED_df = pd.read_pickle(OED_file_path_100gen)
else:
    ngen_100_OL_df, ngen_100_OED_df = extract_best_solutions_max_gen(hybrid_only_files, max_gen=100)
    ngen_100_OL_df.to_pickle(OL_file_path_100gen)
    ngen_100_OED_df.to_pickle(OED_file_path_100gen)
    
ngen_50_OL_df = None
ngen_50_OED_df = None
if OL_file_path_50gen.exists() and OED_file_path_50gen.exists():
    ngen_50_OL_df = pd.read_pickle(OL_file_path_50gen)
    ngen_50_OED_df = pd.read_pickle(OED_file_path_50gen)
else:
    ngen_50_OL_df, ngen_50_OED_df = extract_best_solutions_max_gen(hybrid_only_files, max_gen=50)
    ngen_50_OL_df.to_pickle(OL_file_path_50gen)
    ngen_50_OED_df.to_pickle(OED_file_path_50gen)

CPU times: user 2.26 ms, sys: 1.86 ms, total: 4.12 ms
Wall time: 3.27 ms


In [42]:
hybrid_search_settings = [k for k in better_options.keys() if k not in [FIXED, NONHYBRID]]
hybrid_search_settings

['', 'Ngen_50', 'Ngen_100', 'N_100', 'N_200', 'init_pop_20', 'opt_within_Q2']

## All - Theoretical

In [43]:
all_vals_df = pd.concat([pd.read_pickle(OL_file_path), ngen_100_OL_df, ngen_50_OL_df])
all_vals_df["repair"] = all_vals_df.problem.apply(lambda p: p in repair_circuits)

# Hybrid init_pop and Hybrid are the same for all repair use cases. 
# all_vals_df[all_vals_df.repair & (all_vals_df.option == HYBRID)]

hybrid_init_pop_repair = all_vals_df[all_vals_df.repair & (all_vals_df.option == HYBRID)].copy()
hybrid_init_pop_repair["option"] = "init_pop_20"

all_vals_df = pd.concat([all_vals_df, hybrid_init_pop_repair])

In [44]:
optimization_df = all_vals_df  #[all_vals_df.repair == False].reset_index()
optimization_df["Categorisation_OL"] = optimization_df.apply(get_operator_categorisation_OL, axis=1)

#### SUM

In [45]:
tables = {}
for option in hybrid_search_settings:
    print(better_options[option])
    rows = []
    for problem in optimization_df.problem.unique():
        # print(problem)
        qubits=optimization_df[optimization_df.problem == problem].qubits.iloc[0]
        arbitrary = optimization_df[optimization_df.problem == problem].arbitrary.iloc[0]
        if arbitrary:
            arb = "Arbitrary"
        else:
            arb = "Specific"
        row = dict(problem = problem, qubits=qubits, Arbitrary=arb)

        for which in ["Optimized", "Pareto Equal", "Worse", "Faulty"]:
            row[which] = len(optimization_df[
                (optimization_df.option == option) & 
                (optimization_df.problem == problem) & 
                (optimization_df.Categorisation_OL == which)                
            ])
        rows.append(row)
        
    tab = pd.DataFrame(rows)
    # tab = tab.reindex(tab["Arbitrary"])
    tables[option] = tab
    
    # tab = tab.sum().drop(columns=["problem", "qubits"])
    # tables[option] = pd.DataFrame(tab)
    
for option, tab in tables.items():
    tab = tab.sum()
    tab = pd.DataFrame(tab).T
    
    tab[better_options[option]] = tab.apply(lambda row: " / ".join(row[["Optimized", "Pareto Equal", "Worse", "Faulty"]].astype(str)), axis=1)
    tab = tab.drop(columns=["Optimized", "Pareto Equal", "Worse", "Faulty"])
    tab = tab.sort_values(["qubits", "problem"])  
    tab = tab.drop(columns=["problem", "qubits", "Arbitrary"])    

    tables[option] = tab
    # display(tab)
    
RQ41_summary_table_sum = pd.concat(tables.values(), axis=1)
RQ41_summary_table_sum

Hybrid
Hybrid_{NGen=50}
Hybrid_{NGen=100}
Hybrid_{N=100}
Hybrid_{N=200}
Hybrid_{Init=20}
Hybrid_{Q2}


,Hybrid,Hybrid_{NGen=50},Hybrid_{NGen=100},Hybrid_{N=100},Hybrid_{N=200},Hybrid_{Init=20},Hybrid_{Q2}
0,717 / 164 / 71 / 458,523 / 191 / 95 / 601,653 / 175 / 81 / 501,868 / 181 / 56 / 305,915 / 187 / 59 / 249,874 / 453 / 31 / 52,776 / 152 / 39 / 443


#### Repair/Optimization

In [46]:
tables = {}
for option in hybrid_search_settings:
    print(better_options[option])
    rows = []
    for problem in optimization_df.problem.unique():
        # print(problem)
        qubits=optimization_df[optimization_df.problem == problem].qubits.iloc[0]
        arbitrary = optimization_df[optimization_df.problem == problem].arbitrary.iloc[0]
        is_repair = optimization_df[optimization_df.problem == problem].repair.iloc[0]
        if arbitrary:
            arb = "Arbitrary"
        else:
            arb = "Specific"
        row = dict(problem = problem, qubits=qubits, Arbitrary=arb, repair=("Repair" if is_repair else "Optimization"))

        for which in ["Optimized", "Pareto Equal", "Worse", "Faulty"]:
            row[which] = len(optimization_df[
                (optimization_df.option == option) & 
                (optimization_df.problem == problem) & 
                (optimization_df.Categorisation_OL == which)                
            ])
        rows.append(row)
        
    tab = pd.DataFrame(rows)
    # tab = tab.reindex(tab["Arbitrary"])
    tables[option] = tab
    
    # tab = tab.sum().drop(columns=["problem", "qubits"])
    # tables[option] = pd.DataFrame(tab)
    
for option, tab in tables.items():
    tab = tab.groupby("repair").sum()
    tab[better_options[option]] = tab.apply(lambda row: " / ".join(row[["Optimized", "Pareto Equal", "Worse", "Faulty"]].astype(str)), axis=1)
    tab = tab.drop(columns=["Optimized", "Pareto Equal", "Worse", "Faulty"])
    tab = tab.sort_values(["qubits", "problem"])  
    tab = tab.drop(columns=["problem", "qubits", "Arbitrary"])    

    tables[option] = tab
    # display(tab)
    
RQ41_summary_table_repair = pd.concat(tables.values(), axis=1)
RQ41_summary_table_repair

Hybrid
Hybrid_{NGen=50}
Hybrid_{NGen=100}
Hybrid_{N=100}
Hybrid_{N=200}
Hybrid_{Init=20}
Hybrid_{Q2}


,Hybrid,Hybrid_{NGen=50},Hybrid_{NGen=100},Hybrid_{N=100},Hybrid_{N=200},Hybrid_{Init=20},Hybrid_{Q2}
repair,,,,,,,
Repair,176 / 21 / 21 / 52,162 / 28 / 24 / 56,170 / 26 / 20 / 54,180 / 34 / 25 / 31,180 / 38 / 34 / 18,176 / 21 / 21 / 52,178 / 21 / 21 / 50
Optimization,541 / 143 / 50 / 406,361 / 163 / 71 / 545,483 / 149 / 61 / 447,688 / 147 / 31 / 274,735 / 149 / 25 / 231,698 / 432 / 10 / 0,598 / 131 / 18 / 393


#### Input State

In [47]:
tables = {}
for option in hybrid_search_settings:
    print(better_options[option])
    rows = []
    for problem in optimization_df.problem.unique():
        # print(problem)
        qubits=optimization_df[optimization_df.problem == problem].qubits.iloc[0]
        arbitrary = optimization_df[optimization_df.problem == problem].arbitrary.iloc[0]
        if arbitrary:
            arb = "Arbitrary"
        else:
            arb = "Specific"
        row = dict(problem = problem, qubits=qubits, Arbitrary=arb)

        for which in ["Optimized", "Pareto Equal", "Worse", "Faulty"]:
            row[which] = len(optimization_df[
                (optimization_df.option == option) & 
                (optimization_df.problem == problem) & 
                (optimization_df.Categorisation_OL == which)                
            ])
        rows.append(row)
        
    tab = pd.DataFrame(rows)
    # tab = tab.reindex(tab["Arbitrary"])
    tables[option] = tab
    
    # tab = tab.sum().drop(columns=["problem", "qubits"])
    # tables[option] = pd.DataFrame(tab)
    
for option, tab in tables.items():
    tab = tab.groupby("Arbitrary").sum()
    # display(tab)
    tab[better_options[option]] = tab.apply(lambda row: " / ".join(row[["Optimized", "Pareto Equal", "Worse", "Faulty"]].astype(str)), axis=1)
    tab = tab.drop(columns=["Optimized", "Pareto Equal", "Worse", "Faulty"])
    tab = tab.sort_values(["qubits", "problem"])  
    tab = tab.drop(columns=["problem", "qubits"])    

    tables[option] = tab
    # display(tab)
    
RQ41_summary_table_input_state = pd.concat(tables.values(), axis=1)
RQ41_summary_table_input_state

Hybrid
Hybrid_{NGen=50}
Hybrid_{NGen=100}
Hybrid_{N=100}
Hybrid_{N=200}
Hybrid_{Init=20}
Hybrid_{Q2}


,Hybrid,Hybrid_{NGen=50},Hybrid_{NGen=100},Hybrid_{N=100},Hybrid_{N=200},Hybrid_{Init=20},Hybrid_{Q2}
Arbitrary,,,,,,,
Specific,420 / 93 / 23 / 124,353 / 101 / 33 / 173,397 / 96 / 31 / 136,488 / 95 / 14 / 63,506 / 100 / 11 / 43,501 / 159 / 0 / 0,447 / 90 / 8 / 115
Arbitrary,297 / 71 / 48 / 334,170 / 90 / 62 / 428,256 / 79 / 50 / 365,380 / 86 / 42 / 242,409 / 87 / 48 / 206,373 / 294 / 31 / 52,329 / 62 / 31 / 328


#### Qubits

In [48]:
tables = {}
for option in hybrid_search_settings:
    print(better_options[option])
    rows = []
    for problem in optimization_df.problem.unique():
        # print(problem)
        qubits=optimization_df[optimization_df.problem == problem].qubits.iloc[0]
        arbitrary = optimization_df[optimization_df.problem == problem].arbitrary.iloc[0]
        if arbitrary:
            arb = "Arbitrary"
        else:
            arb = "Specific"
        row = dict(problem = problem, qubits=qubits, Arbitrary=arb)

        for which in ["Optimized", "Pareto Equal", "Worse", "Faulty"]:
            row[which] = len(optimization_df[
                (optimization_df.option == option) & 
                (optimization_df.problem == problem) & 
                (optimization_df.Categorisation_OL == which)                
            ])
        rows.append(row)
        
    tab = pd.DataFrame(rows)
    tables[option] = tab
    
for option, tab in tables.items():
    tab["qubits"] = tab.qubits.astype(str) + " qubits"
    tab = tab.groupby("qubits").sum()
    
    tab[better_options[option]] = tab.apply(lambda row: " / ".join(row[["Optimized", "Pareto Equal", "Worse", "Faulty"]].astype(str)), axis=1)
    tab = tab.drop(columns=["Optimized", "Pareto Equal", "Worse", "Faulty"])
    tab = tab.sort_values(["qubits", "problem"])
    tab = tab.drop(columns=["problem", "Arbitrary"])    
    tables[option] = tab
    
    # display(tab)
    
RQ41_summary_table_qubits = pd.concat(tables.values(), axis=1)
RQ41_summary_table_qubits

Hybrid
Hybrid_{NGen=50}
Hybrid_{NGen=100}
Hybrid_{N=100}
Hybrid_{N=200}
Hybrid_{Init=20}
Hybrid_{Q2}


,Hybrid,Hybrid_{NGen=50},Hybrid_{NGen=100},Hybrid_{N=100},Hybrid_{N=200},Hybrid_{Init=20},Hybrid_{Q2}
qubits,,,,,,,
2 qubits,221 / 44 / 19 / 16,195 / 52 / 28 / 25,209 / 50 / 20 / 21,240 / 45 / 11 / 4,245 / 46 / 8 / 1,241 / 49 / 10 / 0,233 / 41 / 9 / 17
3 qubits,278 / 54 / 17 / 71,205 / 58 / 29 / 128,256 / 57 / 19 / 88,315 / 62 / 7 / 36,335 / 56 / 6 / 23,286 / 121 / 8 / 5,299 / 47 / 8 / 66
4 qubits,150 / 55 / 21 / 164,80 / 73 / 29 / 208,130 / 57 / 30 / 173,217 / 55 / 17 / 101,231 / 52 / 18 / 89,197 / 163 / 7 / 23,180 / 47 / 12 / 151
5 qubits,68 / 11 / 14 / 207,43 / 8 / 9 / 240,58 / 11 / 12 / 219,96 / 19 / 21 / 164,104 / 33 / 27 / 136,150 / 120 / 6 / 24,64 / 17 / 10 / 209


## All - Practical

In [49]:
all_vals_df = pd.concat([pd.read_pickle(OED_file_path), ngen_100_OED_df, ngen_50_OED_df])

all_vals_df["repair"] = all_vals_df.problem.apply(lambda p: p in repair_circuits)

hybrid_init_pop_repair = all_vals_df[all_vals_df.repair & (all_vals_df.option == HYBRID)].copy()
hybrid_init_pop_repair["option"] = "init_pop_20"

all_vals_df = pd.concat([all_vals_df, hybrid_init_pop_repair])

In [50]:
optimization_df = all_vals_df  #[all_vals_df.repair == False].reset_index()
optimization_df["Categorisation_OED"] = optimization_df.apply(get_operator_categorisation_OED, axis=1)

#### SUM

In [51]:
tables = {}
for option in hybrid_search_settings:
    print(better_options[option])
    rows = []
    for problem in optimization_df.problem.unique():
        # print(problem)
        qubits=optimization_df[optimization_df.problem == problem].qubits.iloc[0]
        arbitrary = optimization_df[optimization_df.problem == problem].arbitrary.iloc[0]
        if arbitrary:
            arb = "Arbitrary"
        else:
            arb = "Specific"
        row = dict(problem = problem, qubits=qubits, Arbitrary=arb)

        for which in ["Optimized", "Pareto Equal", "Worse", "Faulty"]:
            row[which] = len(optimization_df[
                (optimization_df.option == option) & 
                (optimization_df.problem == problem) & 
                (optimization_df.Categorisation_OED == which)                
            ])
        rows.append(row)
        
    tab = pd.DataFrame(rows)
    # tab = tab.reindex(tab["Arbitrary"])
    tables[option] = tab
    
    # tab = tab.sum().drop(columns=["problem", "qubits"])
    # tables[option] = pd.DataFrame(tab)
    
for option, tab in tables.items():
    tab = tab.sum()
    tab = pd.DataFrame(tab).T
    
    tab[better_options[option]] = tab.apply(lambda row: " / ".join(row[["Optimized", "Pareto Equal", "Worse", "Faulty"]].astype(str)), axis=1)
    tab = tab.drop(columns=["Optimized", "Pareto Equal", "Worse", "Faulty"])
    tab = tab.sort_values(["qubits", "problem"])  
    tab = tab.drop(columns=["problem", "qubits", "Arbitrary"])    

    tables[option] = tab
    # display(tab)
    
RQ42_summary_table_sum = pd.concat(tables.values(), axis=1)
RQ42_summary_table_sum

Hybrid
Hybrid_{NGen=50}
Hybrid_{NGen=100}
Hybrid_{N=100}
Hybrid_{N=200}
Hybrid_{Init=20}
Hybrid_{Q2}


,Hybrid,Hybrid_{NGen=50},Hybrid_{NGen=100},Hybrid_{N=100},Hybrid_{N=200},Hybrid_{Init=20},Hybrid_{Q2}
0,892 / 105 / 3 / 410,683 / 102 / 4 / 621,820 / 102 / 5 / 483,1055 / 129 / 0 / 226,1111 / 121 / 1 / 177,988 / 370 / 1 / 51,953 / 79 / 2 / 376


#### Repair/Optimization

In [52]:
tables = {}
for option in hybrid_search_settings:
    print(better_options[option])
    rows = []
    for problem in optimization_df.problem.unique():
        # print(problem)
        qubits=optimization_df[optimization_df.problem == problem].qubits.iloc[0]
        arbitrary = optimization_df[optimization_df.problem == problem].arbitrary.iloc[0]
        repair = optimization_df[optimization_df.problem == problem].repair.iloc[0]
        if arbitrary:
            arb = "Arbitrary"
        else:
            arb = "Specific"
        row = dict(problem = problem, qubits=qubits, Arbitrary=arb, repair="Repair" if repair else "Optimization")

        for which in ["Optimized", "Pareto Equal", "Worse", "Faulty"]:
            row[which] = len(optimization_df[
                (optimization_df.option == option) & 
                (optimization_df.problem == problem) & 
                (optimization_df.Categorisation_OED == which)                
            ])
        rows.append(row)
        
    tab = pd.DataFrame(rows)
    # tab = tab.reindex(tab["Arbitrary"])
    tables[option] = tab
    
    # tab = tab.sum().drop(columns=["problem", "qubits"])
    # tables[option] = pd.DataFrame(tab)
    
for option, tab in tables.items():
    tab = tab.groupby("repair").sum()
    tab[better_options[option]] = tab.apply(lambda row: " / ".join(row[["Optimized", "Pareto Equal", "Worse", "Faulty"]].astype(str)), axis=1)
    tab = tab.drop(columns=["Optimized", "Pareto Equal", "Worse", "Faulty"])
    tab = tab.sort_values(["qubits", "problem"])  
    tab = tab.drop(columns=["problem", "qubits", "Arbitrary"])    

    tables[option] = tab
    # display(tab)
    
RQ42_summary_table_repair = pd.concat(tables.values(), axis=1)
RQ42_summary_table_repair

Hybrid
Hybrid_{NGen=50}
Hybrid_{NGen=100}
Hybrid_{N=100}
Hybrid_{N=200}
Hybrid_{Init=20}
Hybrid_{Q2}


,Hybrid,Hybrid_{NGen=50},Hybrid_{NGen=100},Hybrid_{N=100},Hybrid_{N=200},Hybrid_{Init=20},Hybrid_{Q2}
repair,,,,,,,
Repair,191 / 27 / 1 / 51,165 / 31 / 3 / 71,181 / 29 / 2 / 58,206 / 53 / 0 / 11,228 / 40 / 0 / 2,191 / 27 / 1 / 51,209 / 9 / 2 / 50
Optimization,701 / 78 / 2 / 359,518 / 71 / 1 / 550,639 / 73 / 3 / 425,849 / 76 / 0 / 215,883 / 81 / 1 / 175,797 / 343 / 0 / 0,744 / 70 / 0 / 326


#### Input State

In [53]:
tables = {}
for option in hybrid_search_settings:
    print(better_options[option])
    rows = []
    for problem in optimization_df.problem.unique():
        # print(problem)
        qubits=optimization_df[optimization_df.problem == problem].qubits.iloc[0]
        arbitrary = optimization_df[optimization_df.problem == problem].arbitrary.iloc[0]
        if arbitrary:
            arb = "Arbitrary"
        else:
            arb = "Specific"
        row = dict(problem = problem, qubits=qubits, Arbitrary=arb)

        for which in ["Optimized", "Pareto Equal", "Worse", "Faulty"]:
            row[which] = len(optimization_df[
                (optimization_df.option == option) & 
                (optimization_df.problem == problem) & 
                (optimization_df.Categorisation_OED == which)                
            ])
        rows.append(row)
        
    tab = pd.DataFrame(rows)
    # tab = tab.reindex(tab["Arbitrary"])
    tables[option] = tab
    
    # tab = tab.sum().drop(columns=["problem", "qubits"])
    # tables[option] = pd.DataFrame(tab)
    
for option, tab in tables.items():
    tab = tab.groupby("Arbitrary").sum()
    # display(tab)
    tab[better_options[option]] = tab.apply(lambda row: " / ".join(row[["Optimized", "Pareto Equal", "Worse", "Faulty"]].astype(str)), axis=1)
    tab = tab.drop(columns=["Optimized", "Pareto Equal", "Worse", "Faulty"])
    tab = tab.sort_values(["qubits", "problem"])  
    tab = tab.drop(columns=["problem", "qubits"])    

    tables[option] = tab
    # display(tab)
    
RQ42_summary_table_input_state = pd.concat(tables.values(), axis=1)
RQ42_summary_table_input_state

Hybrid
Hybrid_{NGen=50}
Hybrid_{NGen=100}
Hybrid_{N=100}
Hybrid_{N=200}
Hybrid_{Init=20}
Hybrid_{Q2}


,Hybrid,Hybrid_{NGen=50},Hybrid_{NGen=100},Hybrid_{N=100},Hybrid_{N=200},Hybrid_{Init=20},Hybrid_{Q2}
Arbitrary,,,,,,,
Specific,441 / 72 / 2 / 145,371 / 78 / 3 / 208,416 / 74 / 2 / 168,512 / 68 / 0 / 80,526 / 70 / 1 / 63,502 / 158 / 0 / 0,463 / 68 / 0 / 129
Arbitrary,451 / 33 / 1 / 265,312 / 24 / 1 / 413,404 / 28 / 3 / 315,543 / 61 / 0 / 146,585 / 51 / 0 / 114,486 / 212 / 1 / 51,490 / 11 / 2 / 247


#### Qubits

In [54]:
tables = {}
for option in hybrid_search_settings:
    print(better_options[option])
    rows = []
    for problem in optimization_df.problem.unique():
        # print(problem)
        qubits=optimization_df[optimization_df.problem == problem].qubits.iloc[0]
        arbitrary = optimization_df[optimization_df.problem == problem].arbitrary.iloc[0]
        if arbitrary:
            arb = "Arbitrary"
        else:
            arb = "Specific"
        row = dict(problem = problem, qubits=qubits, Arbitrary=arb)

        for which in ["Optimized", "Pareto Equal", "Worse", "Faulty"]:
            row[which] = len(optimization_df[
                (optimization_df.option == option) & 
                (optimization_df.problem == problem) & 
                (optimization_df.Categorisation_OED == which)                
            ])
        rows.append(row)
        
    tab = pd.DataFrame(rows)
    tables[option] = tab
    
for option, tab in tables.items():
    tab["qubits"] = tab.qubits.astype(str) + " qubits"
    tab = tab.groupby("qubits").sum()
    
    tab[better_options[option]] = tab.apply(lambda row: " / ".join(row[["Optimized", "Pareto Equal", "Worse", "Faulty"]].astype(str)), axis=1)
    tab = tab.drop(columns=["Optimized", "Pareto Equal", "Worse", "Faulty"])
    tab = tab.sort_values(["qubits", "problem"])
    tab = tab.drop(columns=["problem", "Arbitrary"])    
    tables[option] = tab
    
    # display(tab)
    
RQ42_summary_table_qubits = pd.concat(tables.values(), axis=1)
RQ42_summary_table_qubits

Hybrid
Hybrid_{NGen=50}
Hybrid_{NGen=100}
Hybrid_{N=100}
Hybrid_{N=200}
Hybrid_{Init=20}
Hybrid_{Q2}


,Hybrid,Hybrid_{NGen=50},Hybrid_{NGen=100},Hybrid_{N=100},Hybrid_{N=200},Hybrid_{Init=20},Hybrid_{Q2}
qubits,,,,,,,
2 qubits,248 / 41 / 0 / 11,226 / 49 / 2 / 23,239 / 46 / 0 / 15,266 / 34 / 0 / 0,270 / 30 / 0 / 0,259 / 41 / 0 / 0,260 / 34 / 0 / 6
3 qubits,333 / 17 / 1 / 69,245 / 18 / 2 / 155,303 / 19 / 3 / 95,373 / 12 / 0 / 35,394 / 8 / 0 / 18,319 / 99 / 0 / 2,351 / 6 / 2 / 61
4 qubits,199 / 39 / 2 / 150,134 / 31 / 0 / 225,181 / 33 / 1 / 175,263 / 56 / 0 / 71,286 / 59 / 0 / 45,230 / 134 / 1 / 25,229 / 38 / 0 / 123
5 qubits,112 / 8 / 0 / 180,78 / 4 / 0 / 218,97 / 4 / 1 / 198,153 / 27 / 0 / 120,161 / 24 / 1 / 114,180 / 96 / 0 / 24,113 / 1 / 0 / 186


## Merge Tables

In [55]:
RQ41_summary_table_sum_out = RQ41_summary_table_sum.copy()
RQ41_summary_table_repair_out = RQ41_summary_table_repair.copy()
RQ41_summary_table_input_state_out = RQ41_summary_table_input_state.copy()
RQ41_summary_table_qubits_out = RQ41_summary_table_qubits.copy()

RQ42_summary_table_sum_out = RQ42_summary_table_sum.copy()
RQ42_summary_table_repair_out = RQ42_summary_table_repair.copy()
RQ42_summary_table_input_state_out = RQ42_summary_table_input_state.copy()
RQ42_summary_table_qubits_out = RQ42_summary_table_qubits.copy()

RQ41_summary_concat = pd.concat([RQ41_summary_table_sum_out, RQ41_summary_table_repair_out, RQ41_summary_table_input_state_out, RQ41_summary_table_qubits_out])
RQ41_summary_concat.columns = pd.MultiIndex.from_product([["RQ4.1 (Perfect Accuracy)"], RQ41_summary_concat.columns])

RQ42_summary_concat = pd.concat([RQ42_summary_table_sum_out, RQ42_summary_table_repair_out, RQ42_summary_table_input_state_out, RQ42_summary_table_qubits_out])
RQ42_summary_concat.columns = pd.MultiIndex.from_product([["RQ2.2 (Acceptable Accuracy)"], RQ42_summary_concat.columns])

RQ4_summary_out = pd.concat([RQ41_summary_concat, RQ42_summary_concat], axis=1)
RQ4_summary_out.index = ["Total", "Repair", "Optimization", "Specific", "Arbitrary", "2 qubits", "3 qubits", "4 qubits", "5 qubits"]
# RQ4_summary_out = RQ4_summary_out.reindex(["Total", "Repair", "Optimization", "Specific", "Arbitrary", "2 qubits", "3 qubits", "4 qubits", "5 qubits"])
display(RQ4_summary_out)


print("Perfect Accuracy")
RQ41_summary_concat_out = pd.concat([RQ41_summary_table_sum_out, RQ41_summary_table_repair_out, RQ41_summary_table_input_state_out, RQ41_summary_table_qubits_out])
RQ41_summary_concat_out.index = ["Total", "Repair", "Optimization", "Specific", "Arbitrary", "2 qubits", "3 qubits", "4 qubits", "5 qubits"]
# RQ41_summary_concat_out = RQ41_summary_concat_out.reindex(["Total", "Repair", "Optimization", "Specific", "Arbitrary", "2 qubits", "3 qubits", "4 qubits", "5 qubits"])
display(RQ41_summary_concat_out)

print("Acceptable Accuracy")
RQ42_summary_concat_out = pd.concat([RQ42_summary_table_sum_out, RQ42_summary_table_repair_out, RQ42_summary_table_input_state_out, RQ42_summary_table_qubits_out])
RQ42_summary_concat_out.index = ["Total", "Repair", "Optimization", "Specific", "Arbitrary", "2 qubits", "3 qubits", "4 qubits", "5 qubits"]
# RQ42_summary_concat_out = RQ42_summary_concat_out.reindex(["Total", "Repair", "Optimization", "Specific", "Arbitrary", "2 qubits", "3 qubits", "4 qubits", "5 qubits"])
display(RQ42_summary_concat_out)


write_table_to_file(RQ41_summary_concat_out, paper_tables_dir / f"{DATA_PREFIX}-RQ4-summary-perfect.tex", col_fix=True, space_slash=False)
write_table_to_file(RQ42_summary_concat_out, paper_tables_dir / f"{DATA_PREFIX}-RQ4-summary-acceptable.tex", col_fix=True, space_slash=False )
write_table_to_file(RQ4_summary_out, paper_tables_dir / f"{DATA_PREFIX}-RQ4-summary.tex", col_fix=True, space_slash=False )

RQ4.1 (Perfect Accuracy)                         
                               Hybrid      Hybrid_{NGen=50}   
Total            717 / 164 / 71 / 458  523 / 191 / 95 / 601  \
Repair             176 / 21 / 21 / 52    162 / 28 / 24 / 56   
Optimization     541 / 143 / 50 / 406  361 / 163 / 71 / 545   
Specific          420 / 93 / 23 / 124  353 / 101 / 33 / 173   
Arbitrary         297 / 71 / 48 / 334   170 / 90 / 62 / 428   
2 qubits           221 / 44 / 19 / 16    195 / 52 / 28 / 25   
3 qubits           278 / 54 / 17 / 71   205 / 58 / 29 / 128   
4 qubits          150 / 55 / 21 / 164    80 / 73 / 29 / 208   
5 qubits           68 / 11 / 14 / 207      43 / 8 / 9 / 240   

                                                           
                 Hybrid_{NGen=100}        Hybrid_{N=100}   
Total         653 / 175 / 81 / 501  868 / 181 / 56 / 305  \
Repair          170 / 26 / 20 / 54    180 / 34 / 25 / 31   
Optimization  483 / 149 / 61 / 447  688 / 147 / 31 / 274   
Specific       397 / 96 / 31 / 136    488 / 95 / 14 / 63   
Arbitrary      256 / 79 / 50 / 365   380 / 86 / 42 / 242   
2 qubits        209 / 50 / 20 / 21     240 / 45 / 11 / 4   
3 qubits        256 / 57 / 19 / 88     315 / 62 / 7 / 36   
4 qubits       130 / 57 / 30 / 173   217 / 55 / 17 / 101   
5 qubits        58 / 11 / 12 / 219    96 / 19 / 21 / 164   

                                                                                
                    Hybrid_{N=200}     Hybrid_{Init=20}           Hybrid_{Q2}   
Total         915 / 187 / 59 / 249  874 / 453 / 31 / 52  776 / 152 / 39 / 443  \
Repair          180 / 38 / 34 / 18   176 / 21 / 21 / 52    178 / 21 / 21 / 50   
Optimization  735 / 149 / 25 / 231   698 / 432 / 10 / 0  598 / 131 / 18 / 393   
Specific       506 / 100 / 11 / 43    501 / 159 / 0 / 0    447 / 90 / 8 / 115   
Arbitrary      409 / 87 / 48 / 206  373 / 294 / 31 / 52   329 / 62 / 31 / 328   
2 qubits          245 / 46 / 8 / 1    241 / 49 / 10 / 0     233 / 41 / 9 / 17   
3 qubits         335 / 56 / 6 / 23    286 / 121 / 8 / 5     299 / 47 / 8 / 66   
4 qubits        231 / 52 / 18 / 89   197 / 163 / 7 / 23   180 / 47 / 12 / 151   
5 qubits       104 / 33 / 27 / 136   150 / 120 / 6 / 24    64 / 17 / 10 / 209   

             RQ2.2 (Acceptable Accuracy)                        
                                  Hybrid     Hybrid_{NGen=50}   
Total                892 / 105 / 3 / 410  683 / 102 / 4 / 621  \
Repair                 191 / 27 / 1 / 51    165 / 31 / 3 / 71   
Optimization          701 / 78 / 2 / 359   518 / 71 / 1 / 550   
Specific              441 / 72 / 2 / 145   371 / 78 / 3 / 208   
Arbitrary             451 / 33 / 1 / 265   312 / 24 / 1 / 413   
2 qubits               248 / 41 / 0 / 11    226 / 49 / 2 / 23   
3 qubits               333 / 17 / 1 / 69   245 / 18 / 2 / 155   
4 qubits              199 / 39 / 2 / 150   134 / 31 / 0 / 225   
5 qubits               112 / 8 / 0 / 180     78 / 4 / 0 / 218   

                                                                                
                Hybrid_{NGen=100}        Hybrid_{N=100}        Hybrid_{N=200}   
Total         820 / 102 / 5 / 483  1055 / 129 / 0 / 226  1111 / 121 / 1 / 177  \
Repair          181 / 29 / 2 / 58     206 / 53 / 0 / 11      228 / 40 / 0 / 2   
Optimization   639 / 73 / 3 / 425    849 / 76 / 0 / 215    883 / 81 / 1 / 175   
Specific       416 / 74 / 2 / 168     512 / 68 / 0 / 80     526 / 70 / 1 / 63   
Arbitrary      404 / 28 / 3 / 315    543 / 61 / 0 / 146    585 / 51 / 0 / 114   
2 qubits        239 / 46 / 0 / 15      266 / 34 / 0 / 0      270 / 30 / 0 / 0   
3 qubits        303 / 19 / 3 / 95     373 / 12 / 0 / 35      394 / 8 / 0 / 18   
4 qubits       181 / 33 / 1 / 175     263 / 56 / 0 / 71     286 / 59 / 0 / 45   
5 qubits         97 / 4 / 1 / 198    153 / 27 / 0 / 120    161 / 24 / 1 / 114   

                                                      
                Hybrid_{Init=20}         Hybrid_{Q2}  
Total         988 / 370 / 1 / 51  953 / 79 / 2 /

Perfect Accuracy


,Hybrid,Hybrid_{NGen=50},Hybrid_{NGen=100},Hybrid_{N=100},Hybrid_{N=200},Hybrid_{Init=20},Hybrid_{Q2}
Total,717 / 164 / 71 / 458,523 / 191 / 95 / 601,653 / 175 / 81 / 501,868 / 181 / 56 / 305,915 / 187 / 59 / 249,874 / 453 / 31 / 52,776 / 152 / 39 / 443
Repair,176 / 21 / 21 / 52,162 / 28 / 24 / 56,170 / 26 / 20 / 54,180 / 34 / 25 / 31,180 / 38 / 34 / 18,176 / 21 / 21 / 52,178 / 21 / 21 / 50
Optimization,541 / 143 / 50 / 406,361 / 163 / 71 / 545,483 / 149 / 61 / 447,688 / 147 / 31 / 274,735 / 149 / 25 / 231,698 / 432 / 10 / 0,598 / 131 / 18 / 393
Specific,420 / 93 / 23 / 124,353 / 101 / 33 / 173,397 / 96 / 31 / 136,488 / 95 / 14 / 63,506 / 100 / 11 / 43,501 / 159 / 0 / 0,447 / 90 / 8 / 115
Arbitrary,297 / 71 / 48 / 334,170 / 90 / 62 / 428,256 / 79 / 50 / 365,380 / 86 / 42 / 242,409 / 87 / 48 / 206,373 / 294 / 31 / 52,329 / 62 / 31 / 328
2 qubits,221 / 44 / 19 / 16,195 / 52 / 28 / 25,209 / 50 / 20 / 21,240 / 45 / 11 / 4,245 / 46 / 8 / 1,241 / 49 / 10 / 0,233 / 41 / 9 / 17
3 qubits,278 / 54 / 17 / 71,205 / 58 / 29 / 128,256 / 57 / 19 / 88,315 / 62 / 7 / 36,335 / 56 / 6 / 23,286 / 121 / 8 / 5,299 / 47 / 8 / 66
4 qubits,150 / 55 / 21 / 164,80 / 73 / 29 / 208,130 / 57 / 30 / 173,217 / 55 / 17 / 101,231 / 52 / 18 / 89,197 / 163 / 7 / 23,180 / 47 / 12 / 151
5 qubits,68 / 11 / 14 / 207,43 / 8 / 9 / 240,58 / 11 / 12 / 219,96 / 19 / 21 / 164,104 / 33 / 27 / 136,150 / 120 / 6 / 24,64 / 17 / 10 / 209


Acceptable Accuracy


,Hybrid,Hybrid_{NGen=50},Hybrid_{NGen=100},Hybrid_{N=100},Hybrid_{N=200},Hybrid_{Init=20},Hybrid_{Q2}
Total,892 / 105 / 3 / 410,683 / 102 / 4 / 621,820 / 102 / 5 / 483,1055 / 129 / 0 / 226,1111 / 121 / 1 / 177,988 / 370 / 1 / 51,953 / 79 / 2 / 376
Repair,191 / 27 / 1 / 51,165 / 31 / 3 / 71,181 / 29 / 2 / 58,206 / 53 / 0 / 11,228 / 40 / 0 / 2,191 / 27 / 1 / 51,209 / 9 / 2 / 50
Optimization,701 / 78 / 2 / 359,518 / 71 / 1 / 550,639 / 73 / 3 / 425,849 / 76 / 0 / 215,883 / 81 / 1 / 175,797 / 343 / 0 / 0,744 / 70 / 0 / 326
Specific,441 / 72 / 2 / 145,371 / 78 / 3 / 208,416 / 74 / 2 / 168,512 / 68 / 0 / 80,526 / 70 / 1 / 63,502 / 158 / 0 / 0,463 / 68 / 0 / 129
Arbitrary,451 / 33 / 1 / 265,312 / 24 / 1 / 413,404 / 28 / 3 / 315,543 / 61 / 0 / 146,585 / 51 / 0 / 114,486 / 212 / 1 / 51,490 / 11 / 2 / 247
2 qubits,248 / 41 / 0 / 11,226 / 49 / 2 / 23,239 / 46 / 0 / 15,266 / 34 / 0 / 0,270 / 30 / 0 / 0,259 / 41 / 0 / 0,260 / 34 / 0 / 6
3 qubits,333 / 17 / 1 / 69,245 / 18 / 2 / 155,303 / 19 / 3 / 95,373 / 12 / 0 / 35,394 / 8 / 0 / 18,319 / 99 / 0 / 2,351 / 6 / 2 / 61
4 qubits,199 / 39 / 2 / 150,134 / 31 / 0 / 225,181 / 33 / 1 / 175,263 / 56 / 0 / 71,286 / 59 / 0 / 45,230 / 134 / 1 / 25,229 / 38 / 0 / 123
5 qubits,112 / 8 / 0 / 180,78 / 4 / 0 / 218,97 / 4 / 1 / 198,153 / 27 / 0 / 120,161 / 24 / 1 / 114,180 / 96 / 0 / 24,113 / 1 / 0 / 186


'\\begin{tabular}{c|ccccccc|ccccccc}\n\\toprule\n & \\multicolumn{7}{c}{RQ4.1 (Perfect Accuracy)} & \\multicolumn{7}{c}{RQ2.2 (Acceptable Accuracy)} \\\\\n & \\hyb & NGen=50 & NGen=100 & N=100 & N=200 & Init=20 & Q2 & \\hyb & NGen=50 & NGen=100 & N=100 & N=200 & Init=20 & Q2 \\\\\n\\midrule\nTotal & 717/164/71/458 & 523/191/95/601 & 653/175/81/501 & 868/181/56/305 & 915/187/59/249 & 874/453/31/52 & 776/152/39/443 & 892/105/3/410 & 683/102/4/621 & 820/102/5/483 & 1055/129/0/226 & 1111/121/1/177 & 988/370/1/51 & 953/79/2/376 \\\\\n\\hline Repair & 176/21/21/52 & 162/28/24/56 & 170/26/20/54 & 180/34/25/31 & 180/38/34/18 & 176/21/21/52 & 178/21/21/50 & 191/27/1/51 & 165/31/3/71 & 181/29/2/58 & 206/53/0/11 & 228/40/0/2 & 191/27/1/51 & 209/9/2/50 \\\\\nOptimization & 541/143/50/406 & 361/163/71/545 & 483/149/61/447 & 688/147/31/274 & 735/149/25/231 & 698/432/10/0 & 598/131/18/393 & 701/78/2/359 & 518/71/1/550 & 639/73/3/425 & 849/76/0/215 & 883/81/1/175 & 797/343/0/0 & 744/70/0/326 \\\\\n\\h